# !!! Сделай ревью функцию разделителя по персентилям. Могут быть проблемы с колонками у которых сразу несколько в нескольких группах персентилей встречается 0

In [160]:
    #from sklearn.model_selection import train_test_split
#from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree

#from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
#from sklearn.metrics import PredictionErrorDisplay

from sklearn.decomposition import PCA

#from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import datetime
import seaborn as sns
sns.set_palette('muted')
sns.set_color_codes('muted')
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')

from statsmodels.graphics.mosaicplot import mosaic
pd.set_option('display.max_columns', 70)

**downloard the data**

In [161]:
#df_geo = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_spb_transport_fitnes_yandex_cian_metro_population_final_version.csv')
df_geo = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_spb_transport_fitnes_yandex_cian_metro_population_final_version_neighbors.csv')
df_time = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_time_based_features_full.csv')
df_hex_unique = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_hex_unique_with_shares_spb.csv')
df_hex_total = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_hex_total_with_shares_spb.csv')

In [162]:
df_geo.shape, df_time.shape, df_hex_unique.shape, df_hex_total.shape

((3892, 97), (3892, 63), (3892, 68), (3892, 68))

**Drop part of hexagons with total visits lees or equal 150 or unique_visits less or equal 50**

In [163]:
df_hex_total_and_unique = pd.merge(df_hex_unique, df_hex_total,
                                how='left', left_on='h3_9', right_on='h3_9')

In [164]:
avaible_hexes = df_hex_total_and_unique[(df_hex_total_and_unique['unique_visits'] > 50)].h3_9.to_list()
len(avaible_hexes)

1223

In [165]:
avaible_hexes = df_hex_total_and_unique[(df_hex_total_and_unique['total_visits'] > 150)
                                       |(df_hex_total_and_unique['unique_visits'] > 50)].h3_9.to_list()
len(avaible_hexes)

1335

In [166]:
df_hex_total = df_hex_total[df_hex_total['h3_9'].isin(avaible_hexes)]
df_hex_unique = df_hex_unique[df_hex_unique['h3_9'].isin(avaible_hexes)]
df_time = df_time[df_time['h3_9'].isin(avaible_hexes)]
df_geo = df_geo[df_geo['h3_9'].isin(avaible_hexes)]

df_geo.shape, df_time.shape, df_hex_unique.shape, df_hex_total.shape

((1335, 97), (1335, 63), (1335, 68), (1335, 68))

**drop from df_time Stay0_05h_share | Stay0_05h and Stay0_05h_per_unique_share_per_unique | Stay0_05h_per_unique, since after filer on total visits there is no hexes with non zero vales on this param**

In [10]:
df_time.describe()

,avg_duration_hrs,median_duration_hrs,weekday_visits,weekend_visits,Stay0_05h,Stay05_1h,Stay1_3h,Stay3_6h,Stay6_9h,Stay9_15h,Stay15_20h,Stay20_plus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,day,evening,morning,night,weekday_visits_share,weekend_visits_share,Stay0_05h_share,Stay05_1h_share,Stay1_3h_share,Stay3_6h_share,Stay6_9h_share,Stay9_15h_share,Stay15_20h_share,Stay20_plus_share,Friday_share,Monday_share,Saturday_share,Sunday_share,Thursday_share,Tuesday_share,Wednesday_share,day_share,evening_share,morning_share,night_share,avg_duration_hrs_per_unique,median_duration_hrs_per_unique,Stay0_05h_per_unique,Stay05_1h_per_unique,Stay1_3h_per_unique,Stay3_6h_per_unique,Stay6_9h_per_unique,Stay9_15h_per_unique,Stay15_20h_per_unique,Stay20_plus_per_unique,Stay0_05h_per_unique_share_per_unique,Stay05_1h_per_unique_share_per_unique,Stay1_3h_per_unique_share_per_unique,Stay3_6h_per_unique_share_per_unique,Stay6_9h_per_unique_share_per_unique,Stay9_15h_per_unique_share_per_unique,Stay15_20h_per_unique_share_per_unique,Stay20_plus_per_unique_share_per_unique
count,1335.000000,1335.000000,1335.000000,1335.000000,1335.0,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.0,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.0,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.0,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000
mean,5.482323,2.629112,447.673408,241.854682,0.0,43.079401,337.728090,115.337079,71.116105,76.844944,22.023221,23.399251,100.420974,115.659176,121.132584,120.722097,74.565543,79.935581,77.092135,268.731086,152.500375,98.514607,169.782022,0.646208,0.353792,0.0,0.059991,0.503013,0.167064,0.097841,0.107272,0.031180,0.033638,0.140759,0.169316,0.177080,0.176713,0.106096,0.116985,0.113051,0.409290,0.212464,0.153675,0.224571,5.263341,2.081036,0.0,9.585768,66.735581,23.614981,10.083895,8.322097,2.859925,3.726592,0.0,0.070349,0.527311,0.193457,0.083892,0.069454,0.023626,0.031912
std,1.333168,0.663518,278.390100,145.021742,0.0,33.715335,199.400045,72.452116,52.771459,56.573213,17.655505,19.381172,70.314828,68.871869,73.091025,72.578551,49.872566,48.614989,46.527463,149.978001,107.542606,55.580239,137.819237,0.033669,0.033669,0.0,0.028743,0.077677,0.026504,0.027245,0.038663,0.016962,0.020831,0.033176,0.021147,0.022031,0.021711,0.023115,0.017098,0.016071,0.078935,0.055097,0.054684,0.077214,2.039124,0.484818,0.0,9.392249,47.212014,14.118637,6.347189,5.567461,2.547128,3.135760,0.0,0.042301,0.097052,0.056141,0.035993,0.035243,0.018266,0.024859
min,1.861463,1.310000,43.000000,18.000000,0.0,0.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,4.000000,11.000000,0.000000,0.000000,0.000000,33.000000,4.000000,2.000000,1.000000,0.329004,0.189474,0.0,0.000000,0.274854,0.030303,0.000000,0.008130,0.000000,0.000000,0.004950,0.000000,0.042105,0.100917,0.000000,0.000000,0.000000,0.247449,0.009901,0.021053,0.011236,1.921363,1.240000,0.0,0.000000,7.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.266667,0.037037,0.000000,0.000000,0.000000,0.000000
25%,4.576332,2.150000,239.000000,131.500000,0.0,19.000000,185.000000,62.000000,32.000000,33.000000,9.000000,8.000000,47.000000,62.000000,66.000000,65.000000,37.000000,43.500000,43.000000,155.000000,68.000000,59.000000,67.000000,0.629938,0.334735,0.0,0.040562,0.446935,0.151368,0.081205,0.081942,0.019108,0.018620,0.122865,0.156974,0.165317,0.164054,0.094043,0.106907,0.103203,0.348515,0.182182,0.115819,0.171523,3.812320,1.722500,0.0,4.000000,37.000000,13.000000,5.000000,4.000000,1.000000,1.000

In [11]:
df_time.drop(['Stay0_05h_share', 'Stay0_05h_per_unique_share_per_unique',
              'Stay0_05h', 'Stay0_05h_per_unique'], axis=1, inplace = True)

# Transform dataframes for ARL

## df_hex_total dataset

In [12]:
df_hex_total.head(3)

,h3_9,total_visits,car_owners_total,carsharing_user_total,not_citizen_rf_total,healthy_food_inter_total,public_trnsprt_user_total,pharmacies_inter_total,beauty_salon_inter_total,elite_sports_inter_total,sprtswr_ftwr_inter_total,sprt_good_inter_total,trvl_rus_total,trvl_abrd_total,sprt_fit_inter_total,hunting_fishing_inter_total,fashion_inter_total,coffee_inter_total,med_serv_inter_total,education_inter_total,delivery_inter_total,markets_inter_total,no_children_total,age_20_total,age_20_30_total,age_30_40_total,age_40_60_total,age_60_total,male_total,female_total,salary_1_total,salary_2_total,salary_3_total,salary_4_total,salary_5_total,car_owners_total_share,carsharing_user_total_share,not_citizen_rf_total_share,healthy_food_inter_total_share,public_trnsprt_user_total_share,pharmacies_inter_total_share,beauty_salon_inter_total_share,elite_sports_inter_total_share,sprtswr_ftwr_inter_total_share,sprt_good_inter_total_share,trvl_rus_total_share,trvl_abrd_total_share,sprt_fit_inter_total_share,hunting_fishing_inter_total_share,fashion_inter_total_share,coffee_inter_total_share,med_serv_inter_total_share,education_inter_total_share,delivery_inter_total_share,markets_inter_total_share,no_children_total_share,age_20_total_share,age_20_30_total_share,age_30_40_total_share,age_40_60_total_share,age_60_total_share,male_total_share,female_total_share,salary_1_total_share,salary_2_total_share,salary_3_total_share,salary_4_total_share,salary_5_total_share
0,8911060006bffff,147,83,22,57,3,6,60,5,3,1,3,49,6,2,2,8,6,31,8,33,79,136,0,24,34,65,21,91,56,33,40,27,15,18,0.564626,0.149660,0.387755,0.020408,0.040816,0.408163,0.034014,0.020408,0.006803,0.020408,0.333333,0.040816,0.013605,0.013605,0.054422,0.040816,0.210884,0.054422,0.224490,0.537415,0.925170,0.000000,0.166667,0.236111,0.451389,0.145833,0.619048,0.380952,0.248120,0.300752,0.203008,0.112782,0.135338
1,89110600077ffff,250,151,72,133,1,27,78,1,11,1,58,70,34,20,5,15,38,69,31,99,109,189,1,40,90,92,16,101,149,31,39,50,55,39,0.604000,0.288000,0.532000,0.004000,0.108000,0.312000,0.004000,0.044000,0.004000,0.232000,0.280000,0.136000,0.080000,0.020000,0.060000,0.152000,0.276000,0.124000,0.396000,0.436000,0.756000,0.004184,0.167364,0.376569,0.384937,0.066946,0.404000,0.596000,0.144860,0.182243,0.233645,0.257009,0.182243
2,89110600113ffff,333,175,65,204,4,5,177,8,23,0,39,25,3,10,14,94,51,70,75,96,206,257,1,36,165,110,14,191,142,68,62,65,70,50,0.525526,0.195195,0.612613,0.012012,0.015015,0.531532,0.024024,0.069069,0.000000,0.117117,0.075075,0.009009,0.030030,0.042042,0.282282,0.153153,0.210210,0.225225,0.288288,0.618619,0.771772,0.003067,0.110429,0.506135,0.337423,0.042945,0.573574,0.426426,0.215873,0.196825,0.206349,0.222222,0.158730


**we need to think more what to do with columns where there is percentiles with equal values. currently I will just make several 1s during one hot encoding such process**

**function for creating table with percentile borders**

In [13]:
def filter_columns_and_calculate_percentiles(df, drop_strings, percentiles_list, mode='exclude'):
    """
    Filters columns based on the presence or absence of substrings and calculates specified percentiles.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        drop_strings (list of str): Substrings based on which columns are filtered.
        percentiles_list (list of int): Percentiles to calculate for the selected columns.
        mode (str): Determines if 'drop_strings' should be used to 'include' or 'exclude' columns. 
                    'exclude' (default) will exclude columns containing any drop_strings.
                    'include' will include only columns containing any drop_strings.
    
    Returns:
        pd.DataFrame: A DataFrame containing the column names and their respective percentiles.
    """
    if mode == 'exclude':
        filtered_columns = [col for col in df.columns if not any(drop_str in col for drop_str in drop_strings)]
    elif mode == 'include':
        filtered_columns = [col for col in df.columns if any(drop_str in col for drop_str in drop_strings)]
    else:
        raise ValueError("Mode should be either 'exclude' or 'include'.")

    df_filtered = df[filtered_columns]
    
    percentile_data = []
    for col in df_filtered.columns:
        percentiles = np.percentile(df_filtered[col], percentiles_list)
        percentile_data.append([col] + list(percentiles))
    
    column_names = ['column_name'] + [f'{p}_percentile' for p in percentiles_list]
    percentile_df = pd.DataFrame(percentile_data, columns=column_names)
    
    return percentile_df

In [14]:
hex_total_percentiles = filter_columns_and_calculate_percentiles(df_hex_total, ['share', 'h3_9'], [0, 20, 40, 60, 80, 100],
                                                                 mode = 'exclude')
hex_total_percentiles.head(10)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,total_visits,77.0,329.6,499.0,711.8,1028.0,3439.0
1,car_owners_total,8.0,168.8,256.0,371.0,530.0,1914.0
2,carsharing_user_total,1.0,58.0,102.6,154.0,219.0,1164.0
3,not_citizen_rf_total,36.0,162.0,262.6,379.4,553.0,1895.0
4,healthy_food_inter_total,0.0,2.0,6.0,18.0,42.0,224.0
5,public_trnsprt_user_total,0.0,5.0,17.0,31.0,56.0,182.0
6,pharmacies_inter_total,18.0,182.0,288.0,418.0,614.0,2340.0
7,beauty_salon_inter_total,0.0,5.0,16.0,34.0,60.0,292.0
8,elite_sports_inter_total,0.0,0.0,1.0,3.0,13.0,159.0
9,sprtswr_ftwr_inter_total,0.0,0.0,1.0,2.0,12.0,142.0


In [15]:
hex_total_percentiles.shape

(34, 7)

**function for OHE**

1. For each column in df_hex_total that also appears in hex_total_percentiles['column_name'], it created 5 new columns representing the percentile ranges (0_20, 20_40, 40_60, 60_80, 80_100).
2. It then checked in which percentile range the column value falls and marked that range with a 1, leaving the other ranges with 0s.
3. In cases where adjacent percentile values were equal and the column value equaled that as well, it marked multiple ranges with 1s as specified.

In [16]:
def create_adjusted_percentile_columns_with_tracking(hex_total_percentiles, df_hex_total, main_column):
    new_df = df_hex_total[[main_column]].copy()
    # Initialize a list to track columns that get filled in the "if not filled" condition
    not_filled_columns = []
    
    for _, row in hex_total_percentiles.iterrows():
        column_name = row['column_name']
        if column_name in df_hex_total.columns:
            percentiles = row[1:].values
            percentile_ranges = ['0_20', '20_40', '40_60', '60_80', '80_100']
            for pr in percentile_ranges:
                new_col_name = f'{column_name}_{pr}'
                new_df[new_col_name] = 0

            for i, value in df_hex_total.iterrows():
                col_value = value[column_name]
                filled = False
                for j in range(5):
                    low = percentiles[j]
                    high = percentiles[j+1] if j < 4 else percentiles[j]
                    if low < col_value <= high or (j == 0 and col_value == low):
                        new_df.at[i, f'{column_name}_{percentile_ranges[j]}'] = 1
                        filled = True
                    elif col_value == high and j < 4:
                        new_df.at[i, f'{column_name}_{percentile_ranges[j+1]}'] = 1
                        filled = True
                
                if not filled:
                    # If the value is exactly 0 or 1, mark the special case and add to tracking list
                    if col_value == 0:
                        new_df.at[i, f'{column_name}_0_20'] = 1
                        if f'{column_name}_0_20' not in not_filled_columns:
                            not_filled_columns.append(f'{column_name}_0_20')
                    elif col_value == 1:
                        new_df.at[i, f'{column_name}_80_100'] = 1
                        if f'{column_name}_80_100' not in not_filled_columns:
                            not_filled_columns.append(f'{column_name}_80_100')
    
    return new_df, not_filled_columns

**add prcntl ending to mark such columns**

In [17]:
df_hex_total_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(hex_total_percentiles, df_hex_total, "h3_9")
for column in df_hex_total_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_hex_total_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [18]:
wrong_columns_check

[]

In [19]:
df_hex_total_percentile_ARL.shape

(1335, 171)

In [20]:
df_hex_total_percentile_ARL.head(5)

,h3_9,total_visits_0_20_prcntl,total_visits_20_40_prcntl,total_visits_40_60_prcntl,total_visits_60_80_prcntl,total_visits_80_100_prcntl,car_owners_total_0_20_prcntl,car_owners_total_20_40_prcntl,car_owners_total_40_60_prcntl,car_owners_total_60_80_prcntl,car_owners_total_80_100_prcntl,carsharing_user_total_0_20_prcntl,carsharing_user_total_20_40_prcntl,carsharing_user_total_40_60_prcntl,carsharing_user_total_60_80_prcntl,carsharing_user_total_80_100_prcntl,not_citizen_rf_total_0_20_prcntl,not_citizen_rf_total_20_40_prcntl,not_citizen_rf_total_40_60_prcntl,not_citizen_rf_total_60_80_prcntl,not_citizen_rf_total_80_100_prcntl,healthy_food_inter_total_0_20_prcntl,healthy_food_inter_total_20_40_prcntl,healthy_food_inter_total_40_60_prcntl,healthy_food_inter_total_60_80_prcntl,healthy_food_inter_total_80_100_prcntl,public_trnsprt_user_total_0_20_prcntl,public_trnsprt_user_total_20_40_prcntl,public_trnsprt_user_total_40_60_prcntl,public_trnsprt_user_total_60_80_prcntl,public_trnsprt_user_total_80_100_prcntl,pharmacies_inter_total_0_20_prcntl,pharmacies_inter_total_20_40_prcntl,pharmacies_inter_total_40_60_prcntl,pharmacies_inter_total_60_80_prcntl,...,male_total_0_20_prcntl,male_total_20_40_prcntl,male_total_40_60_prcntl,male_total_60_80_prcntl,male_total_80_100_prcntl,female_total_0_20_prcntl,female_total_20_40_prcntl,female_total_40_60_prcntl,female_total_60_80_prcntl,female_total_80_100_prcntl,salary_1_total_0_20_prcntl,salary_1_total_20_40_prcntl,salary_1_total_40_60_prcntl,salary_1_total_60_80_prcntl,salary_1_total_80_100_prcntl,salary_2_total_0_20_prcntl,salary_2_total_20_40_prcntl,salary_2_total_40_60_prcntl,salary_2_total_60_80_prcntl,salary_2_total_80_100_prcntl,salary_3_total_0_20_prcntl,salary_3_total_20_40_prcntl,salary_3_total_40_60_prcntl,salary_3_total_60_80_prcntl,salary_3_total_80_100_prcntl,salary_4_total_0_20_prcntl,salary_4_total_20_40_prcntl,salary_4_total_40_60_prcntl,salary_4_total_60_80_prcntl,salary_4_total_80_100_prcntl,salary_5_total_0_20_prcntl,salary_5_total_20_40_prcntl,salary_5_total_40_60_prcntl,salary_5_total_60_80_prcntl,salary_5_total_80_100_prcntl
0,8911060006bffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,89110600077ffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,89110600113ffff,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
7,8911060018fffff,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
9,8911060019bffff,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [21]:
def create_binary_range_columns(df, main_column, list_of_strings):
    # Initialize the new dataframe with only the main column
    new_df = df[[main_column]].copy()
    
    # Filter columns based on the list_of_strings
    filtered_columns = [col for col in df.columns if any(s in col for s in list_of_strings)]
    
    # Define the range labels and their corresponding numeric boundaries
    range_labels = ['0_20', '20_40', '40_60', '60_80', '80_100']
    range_boundaries = [(0.00, 0.20), (0.20, 0.40), (0.40, 0.60), (0.60, 0.80), (0.80, 1.00)]
    
    # Process each filtered column
    for col in filtered_columns:
        for label, (low, high) in zip(range_labels, range_boundaries):
            # Create a new binary column for the current range
            new_col_name = f'{col}_{label}'
            new_df[new_col_name] = df[col].apply(lambda x: 1 if low < x <= high else 0)
            
            # Adjust for the first range to include 0
            if label == '0_20':
                new_df[new_col_name] = df[col].apply(lambda x: 1 if low <= x <= high else 0)
    
    return new_df

1. For each column in df_hex_total_example that contains strings from the provided list (["total", "owners", "user", "citizen"]), it created 5 new columns to represent the value ranges (0.00-0.20, 0.20-0.40, 0.40-0.60, 0.60-0.80, 0.80-1.00).
2. Each of these new columns is named with the original column name plus the range it represents (e.g., total_visits_0_20).
3. The function then checked if the original column's value falls within each range and marked that range with a 1, leaving others with 0s.

**work with share df with percentile ranges as well**

In [22]:
df_hex_total.head(5)

,h3_9,total_visits,car_owners_total,carsharing_user_total,not_citizen_rf_total,healthy_food_inter_total,public_trnsprt_user_total,pharmacies_inter_total,beauty_salon_inter_total,elite_sports_inter_total,sprtswr_ftwr_inter_total,sprt_good_inter_total,trvl_rus_total,trvl_abrd_total,sprt_fit_inter_total,hunting_fishing_inter_total,fashion_inter_total,coffee_inter_total,med_serv_inter_total,education_inter_total,delivery_inter_total,markets_inter_total,no_children_total,age_20_total,age_20_30_total,age_30_40_total,age_40_60_total,age_60_total,male_total,female_total,salary_1_total,salary_2_total,salary_3_total,salary_4_total,salary_5_total,car_owners_total_share,carsharing_user_total_share,not_citizen_rf_total_share,healthy_food_inter_total_share,public_trnsprt_user_total_share,pharmacies_inter_total_share,beauty_salon_inter_total_share,elite_sports_inter_total_share,sprtswr_ftwr_inter_total_share,sprt_good_inter_total_share,trvl_rus_total_share,trvl_abrd_total_share,sprt_fit_inter_total_share,hunting_fishing_inter_total_share,fashion_inter_total_share,coffee_inter_total_share,med_serv_inter_total_share,education_inter_total_share,delivery_inter_total_share,markets_inter_total_share,no_children_total_share,age_20_total_share,age_20_30_total_share,age_30_40_total_share,age_40_60_total_share,age_60_total_share,male_total_share,female_total_share,salary_1_total_share,salary_2_total_share,salary_3_total_share,salary_4_total_share,salary_5_total_share
0,8911060006bffff,147,83,22,57,3,6,60,5,3,1,3,49,6,2,2,8,6,31,8,33,79,136,0,24,34,65,21,91,56,33,40,27,15,18,0.564626,0.149660,0.387755,0.020408,0.040816,0.408163,0.034014,0.020408,0.006803,0.020408,0.333333,0.040816,0.013605,0.013605,0.054422,0.040816,0.210884,0.054422,0.224490,0.537415,0.925170,0.000000,0.166667,0.236111,0.451389,0.145833,0.619048,0.380952,0.248120,0.300752,0.203008,0.112782,0.135338
1,89110600077ffff,250,151,72,133,1,27,78,1,11,1,58,70,34,20,5,15,38,69,31,99,109,189,1,40,90,92,16,101,149,31,39,50,55,39,0.604000,0.288000,0.532000,0.004000,0.108000,0.312000,0.004000,0.044000,0.004000,0.232000,0.280000,0.136000,0.080000,0.020000,0.060000,0.152000,0.276000,0.124000,0.396000,0.436000,0.756000,0.004184,0.167364,0.376569,0.384937,0.066946,0.404000,0.596000,0.144860,0.182243,0.233645,0.257009,0.182243
2,89110600113ffff,333,175,65,204,4,5,177,8,23,0,39,25,3,10,14,94,51,70,75,96,206,257,1,36,165,110,14,191,142,68,62,65,70,50,0.525526,0.195195,0.612613,0.012012,0.015015,0.531532,0.024024,0.069069,0.000000,0.117117,0.075075,0.009009,0.030030,0.042042,0.282282,0.153153,0.210210,0.225225,0.288288,0.618619,0.771772,0.003067,0.110429,0.506135,0.337423,0.042945,0.573574,0.426426,0.215873,0.196825,0.206349,0.222222,0.158730
7,8911060018fffff,439,235,72,253,5,10,258,16,4,0,117,72,13,5,16,33,53,80,54,133,250,397,3,20,143,213,49,247,192,121,114,88,34,62,0.535308,0.164009,0.576310,0.011390,0.022779,0.587699,0.036446,0.009112,0.000000,0.266515,0.164009,0.029613,0.011390,0.036446,0.075171,0.120729,0.182232,0.123007,0.302961,0.569476,0.904328,0.007009,0.046729,0.334112,0.497664,0.114486,0.562642,0.437358,0.288783,0.272076,0.210024,0.081146,0.147971
9,8911060019bffff,249,123,107,111,3,43,154,42,2,0,39,70,5,0,10,22,14,74,79,114,134,166,3,51,64,113,14,110,139,18,27,52,32,103,0.493976,0.429719,0.445783,0.012048,0.172691,0.618474,0.168675,0.008032,0.000000,0.156627,0.281124,0.020080,0.000000,0.040161,0.088353,0.056225,0.297189,0.317269,0.457831,0.538153,0.666667,0.012245,0.208163,0.261224,0.461224,0.057143,0.441767,0.558233,0.077586,0.116379,0.224138,0.137931,0.443966


In [23]:
hex_total_share_percentiles = filter_columns_and_calculate_percentiles(df_hex_total, ['share'], [0, 20, 40, 60, 80, 100],
                                                                      mode = 'include')
len(hex_total_share_percentiles)

33

In [24]:
hex_total_share_percentiles.head(10)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,car_owners_total_share,0.048485,0.428987,0.493605,0.548413,0.605476,0.891429
1,carsharing_user_total_share,0.006250,0.135825,0.182488,0.227491,0.274963,0.645833
2,not_citizen_rf_total_share,0.180055,0.452160,0.508346,0.556226,0.612625,0.937705
3,healthy_food_inter_total_share,0.000000,0.003764,0.012720,0.030442,0.058228,0.362245
4,public_trnsprt_user_total_share,0.000000,0.011554,0.028180,0.049075,0.079950,0.326531
5,pharmacies_inter_total_share,0.098901,0.500517,0.565664,0.611879,0.668797,0.857426
6,beauty_salon_inter_total_share,0.000000,0.010721,0.029398,0.051221,0.082437,0.350230
7,elite_sports_inter_total_share,0.000000,0.000000,0.002366,0.005650,0.018191,0.190311
8,sprtswr_ftwr_inter_total_share,0.000000,0.000000,0.000807,0.003888,0.016528,0.177632
9,sprt_good_inter_total_share,0.000000,0.079040,0.118771,0.153356,0.199299,0.566667


**add prcntl ending to mark such columns**

In [25]:
df_hex_total_percentile_ARL_shares, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(hex_total_share_percentiles, df_hex_total, "h3_9")
for column in df_hex_total_percentile_ARL_shares.columns:
    if column not in ['h3_9']:
        df_hex_total_percentile_ARL_shares.rename(columns={column: column + '_prcntl'}, inplace=True)

In [26]:
wrong_columns_check

[]

In [27]:
df_hex_total_percentile_ARL_shares.shape

(1335, 166)

In [28]:
df_hex_total_percentile_ARL_shares.head(5)

,h3_9,car_owners_total_share_0_20_prcntl,car_owners_total_share_20_40_prcntl,car_owners_total_share_40_60_prcntl,car_owners_total_share_60_80_prcntl,car_owners_total_share_80_100_prcntl,carsharing_user_total_share_0_20_prcntl,carsharing_user_total_share_20_40_prcntl,carsharing_user_total_share_40_60_prcntl,carsharing_user_total_share_60_80_prcntl,carsharing_user_total_share_80_100_prcntl,not_citizen_rf_total_share_0_20_prcntl,not_citizen_rf_total_share_20_40_prcntl,not_citizen_rf_total_share_40_60_prcntl,not_citizen_rf_total_share_60_80_prcntl,not_citizen_rf_total_share_80_100_prcntl,healthy_food_inter_total_share_0_20_prcntl,healthy_food_inter_total_share_20_40_prcntl,healthy_food_inter_total_share_40_60_prcntl,healthy_food_inter_total_share_60_80_prcntl,healthy_food_inter_total_share_80_100_prcntl,public_trnsprt_user_total_share_0_20_prcntl,public_trnsprt_user_total_share_20_40_prcntl,public_trnsprt_user_total_share_40_60_prcntl,public_trnsprt_user_total_share_60_80_prcntl,public_trnsprt_user_total_share_80_100_prcntl,pharmacies_inter_total_share_0_20_prcntl,pharmacies_inter_total_share_20_40_prcntl,pharmacies_inter_total_share_40_60_prcntl,pharmacies_inter_total_share_60_80_prcntl,pharmacies_inter_total_share_80_100_prcntl,beauty_salon_inter_total_share_0_20_prcntl,beauty_salon_inter_total_share_20_40_prcntl,beauty_salon_inter_total_share_40_60_prcntl,beauty_salon_inter_total_share_60_80_prcntl,...,male_total_share_0_20_prcntl,male_total_share_20_40_prcntl,male_total_share_40_60_prcntl,male_total_share_60_80_prcntl,male_total_share_80_100_prcntl,female_total_share_0_20_prcntl,female_total_share_20_40_prcntl,female_total_share_40_60_prcntl,female_total_share_60_80_prcntl,female_total_share_80_100_prcntl,salary_1_total_share_0_20_prcntl,salary_1_total_share_20_40_prcntl,salary_1_total_share_40_60_prcntl,salary_1_total_share_60_80_prcntl,salary_1_total_share_80_100_prcntl,salary_2_total_share_0_20_prcntl,salary_2_total_share_20_40_prcntl,salary_2_total_share_40_60_prcntl,salary_2_total_share_60_80_prcntl,salary_2_total_share_80_100_prcntl,salary_3_total_share_0_20_prcntl,salary_3_total_share_20_40_prcntl,salary_3_total_share_40_60_prcntl,salary_3_total_share_60_80_prcntl,salary_3_total_share_80_100_prcntl,salary_4_total_share_0_20_prcntl,salary_4_total_share_20_40_prcntl,salary_4_total_share_40_60_prcntl,salary_4_total_share_60_80_prcntl,salary_4_total_share_80_100_prcntl,salary_5_total_share_0_20_prcntl,salary_5_total_share_20_40_prcntl,salary_5_total_share_40_60_prcntl,salary_5_total_share_60_80_prcntl,salary_5_total_share_80_100_prcntl
0,8911060006bffff,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,89110600077ffff,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,89110600113ffff,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
7,8911060018fffff,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
9,8911060019bffff,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0


**making combined df**

In [29]:
df_hex_total_ARL_combined = pd.merge(df_hex_total_percentile_ARL, df_hex_total_percentile_ARL_shares,
                                how='left', left_on='h3_9', right_on='h3_9')

In [30]:
df_hex_total_ARL_combined.shape

(1335, 336)

In [31]:
df_hex_total_ARL_combined.head(3)

,h3_9,total_visits_0_20_prcntl,total_visits_20_40_prcntl,total_visits_40_60_prcntl,total_visits_60_80_prcntl,total_visits_80_100_prcntl,car_owners_total_0_20_prcntl,car_owners_total_20_40_prcntl,car_owners_total_40_60_prcntl,car_owners_total_60_80_prcntl,car_owners_total_80_100_prcntl,carsharing_user_total_0_20_prcntl,carsharing_user_total_20_40_prcntl,carsharing_user_total_40_60_prcntl,carsharing_user_total_60_80_prcntl,carsharing_user_total_80_100_prcntl,not_citizen_rf_total_0_20_prcntl,not_citizen_rf_total_20_40_prcntl,not_citizen_rf_total_40_60_prcntl,not_citizen_rf_total_60_80_prcntl,not_citizen_rf_total_80_100_prcntl,healthy_food_inter_total_0_20_prcntl,healthy_food_inter_total_20_40_prcntl,healthy_food_inter_total_40_60_prcntl,healthy_food_inter_total_60_80_prcntl,healthy_food_inter_total_80_100_prcntl,public_trnsprt_user_total_0_20_prcntl,public_trnsprt_user_total_20_40_prcntl,public_trnsprt_user_total_40_60_prcntl,public_trnsprt_user_total_60_80_prcntl,public_trnsprt_user_total_80_100_prcntl,pharmacies_inter_total_0_20_prcntl,pharmacies_inter_total_20_40_prcntl,pharmacies_inter_total_40_60_prcntl,pharmacies_inter_total_60_80_prcntl,...,male_total_share_0_20_prcntl,male_total_share_20_40_prcntl,male_total_share_40_60_prcntl,male_total_share_60_80_prcntl,male_total_share_80_100_prcntl,female_total_share_0_20_prcntl,female_total_share_20_40_prcntl,female_total_share_40_60_prcntl,female_total_share_60_80_prcntl,female_total_share_80_100_prcntl,salary_1_total_share_0_20_prcntl,salary_1_total_share_20_40_prcntl,salary_1_total_share_40_60_prcntl,salary_1_total_share_60_80_prcntl,salary_1_total_share_80_100_prcntl,salary_2_total_share_0_20_prcntl,salary_2_total_share_20_40_prcntl,salary_2_total_share_40_60_prcntl,salary_2_total_share_60_80_prcntl,salary_2_total_share_80_100_prcntl,salary_3_total_share_0_20_prcntl,salary_3_total_share_20_40_prcntl,salary_3_total_share_40_60_prcntl,salary_3_total_share_60_80_prcntl,salary_3_total_share_80_100_prcntl,salary_4_total_share_0_20_prcntl,salary_4_total_share_20_40_prcntl,salary_4_total_share_40_60_prcntl,salary_4_total_share_60_80_prcntl,salary_4_total_share_80_100_prcntl,salary_5_total_share_0_20_prcntl,salary_5_total_share_20_40_prcntl,salary_5_total_share_40_60_prcntl,salary_5_total_share_60_80_prcntl,salary_5_total_share_80_100_prcntl
0,8911060006bffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,89110600077ffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,89110600113ffff,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0


**making combined percentile df to check rangesin future**

In [32]:
hex_total_share_percentiles.shape, hex_total_percentiles.shape

((33, 7), (34, 7))

In [33]:
df_hex_total_percentile_ranges = pd.concat([hex_total_share_percentiles, hex_total_percentiles], ignore_index=True)

In [34]:
df_hex_total_percentile_ranges.shape

(67, 7)

In [35]:
df_hex_total_percentile_ranges.head(10)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,car_owners_total_share,0.048485,0.428987,0.493605,0.548413,0.605476,0.891429
1,carsharing_user_total_share,0.006250,0.135825,0.182488,0.227491,0.274963,0.645833
2,not_citizen_rf_total_share,0.180055,0.452160,0.508346,0.556226,0.612625,0.937705
3,healthy_food_inter_total_share,0.000000,0.003764,0.012720,0.030442,0.058228,0.362245
4,public_trnsprt_user_total_share,0.000000,0.011554,0.028180,0.049075,0.079950,0.326531
5,pharmacies_inter_total_share,0.098901,0.500517,0.565664,0.611879,0.668797,0.857426
6,beauty_salon_inter_total_share,0.000000,0.010721,0.029398,0.051221,0.082437,0.350230
7,elite_sports_inter_total_share,0.000000,0.000000,0.002366,0.005650,0.018191,0.190311
8,sprtswr_ftwr_inter_total_share,0.000000,0.000000,0.000807,0.003888,0.016528,0.177632
9,sprt_good_inter_total_share,0.000000,0.079040,0.118771,0.153356,0.199299,0.566667


In [36]:
#df_hex_total_percentile_ranges
#df_hex_total_ARL_combined

## df_hex_unique dataset

In [37]:
df_hex_unique.head(3)

,h3_9,unique_visits,car_owners_unique,carsharing_user_unique,not_citizen_rf_unique,healthy_food_inter_unique,public_trnsprt_user_unique,pharmacies_inter_unique,beauty_salon_inter_unique,elite_sports_inter_unique,sprtswr_ftwr_inter_unique,sprt_good_inter_unique,trvl_rus_unique,trvl_abrd_unique,sprt_fit_inter_unique,hunting_fishing_inter_unique,fashion_inter_unique,coffee_inter_unique,med_serv_inter_unique,education_inter_unique,delivery_inter_unique,markets_inter_unique,no_children_unique,age_20_unique,age_20_30_unique,age_30_40_unique,age_40_60_unique,age_60_unique,male_unique,female_unique,salary_1_unique,salary_2_unique,salary_3_unique,salary_4_unique,salary_5_unique,car_owners_unique_share,carsharing_user_unique_share,not_citizen_rf_unique_share,healthy_food_inter_unique_share,public_trnsprt_user_unique_share,pharmacies_inter_unique_share,beauty_salon_inter_unique_share,elite_sports_inter_unique_share,sprtswr_ftwr_inter_unique_share,sprt_good_inter_unique_share,trvl_rus_unique_share,trvl_abrd_unique_share,sprt_fit_inter_unique_share,hunting_fishing_inter_unique_share,fashion_inter_unique_share,coffee_inter_unique_share,med_serv_inter_unique_share,education_inter_unique_share,delivery_inter_unique_share,markets_inter_unique_share,no_children_unique_share,age_20_unique_share,age_20_30_unique_share,age_30_40_unique_share,age_40_60_unique_share,age_60_unique_share,male_unique_share,female_unique_share,salary_1_unique_share,salary_2_unique_share,salary_3_unique_share,salary_4_unique_share,salary_5_unique_share
0,8911060006bffff,98,50,12,44,3,4,38,4,1,1,3,22,4,2,2,5,6,14,8,22,41,90,0,12,29,41,14,64,34,29,15,12,13,16,0.510204,0.122449,0.448980,0.030612,0.040816,0.387755,0.040816,0.010204,0.010204,0.030612,0.224490,0.040816,0.020408,0.020408,0.051020,0.061224,0.142857,0.081633,0.224490,0.418367,0.918367,0.000000,0.125000,0.302083,0.427083,0.145833,0.653061,0.346939,0.341176,0.176471,0.141176,0.152941,0.188235
1,89110600077ffff,99,53,27,48,1,3,42,1,3,1,10,27,8,4,5,4,13,17,15,26,48,83,1,12,26,39,12,69,30,20,13,20,18,16,0.535354,0.272727,0.484848,0.010101,0.030303,0.424242,0.010101,0.030303,0.010101,0.101010,0.272727,0.080808,0.040404,0.050505,0.040404,0.131313,0.171717,0.151515,0.262626,0.484848,0.838384,0.011111,0.133333,0.288889,0.433333,0.133333,0.696970,0.303030,0.229885,0.149425,0.229885,0.206897,0.183908
2,89110600113ffff,144,79,31,80,4,5,66,6,1,0,14,21,3,7,4,17,20,32,28,34,66,118,1,10,51,62,13,92,52,42,26,22,20,19,0.548611,0.215278,0.555556,0.027778,0.034722,0.458333,0.041667,0.006944,0.000000,0.097222,0.145833,0.020833,0.048611,0.027778,0.118056,0.138889,0.222222,0.194444,0.236111,0.458333,0.819444,0.007299,0.072993,0.372263,0.452555,0.094891,0.638889,0.361111,0.325581,0.201550,0.170543,0.155039,0.147287


In [38]:
hex_unique_percentiles = filter_columns_and_calculate_percentiles(df_hex_unique, ['share', 'h3_9'], [0, 20, 40, 60, 80, 100],
                                                                 mode = 'exclude')
hex_unique_percentiles.head(5)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,unique_visits,17.0,66.0,94.0,124.0,172.0,788.0
1,car_owners_unique,6.0,31.0,45.0,59.0,82.0,332.0
2,carsharing_user_unique,1.0,12.0,17.0,24.0,32.0,204.0
3,not_citizen_rf_unique,8.0,33.0,48.0,65.0,88.0,413.0
4,healthy_food_inter_unique,0.0,1.0,2.0,4.0,6.0,30.0


In [39]:
hex_unique_percentiles.shape

(34, 7)

**add prcntl ending to mark such columns**

In [40]:
df_hex_unique_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(hex_unique_percentiles, df_hex_unique, "h3_9")
for column in df_hex_unique_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_hex_unique_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [41]:
wrong_columns_check

[]

In [42]:
df_hex_unique_percentile_ARL.shape

(1335, 171)

In [43]:
df_hex_unique_percentile_ARL.head(5)

,h3_9,unique_visits_0_20_prcntl,unique_visits_20_40_prcntl,unique_visits_40_60_prcntl,unique_visits_60_80_prcntl,unique_visits_80_100_prcntl,car_owners_unique_0_20_prcntl,car_owners_unique_20_40_prcntl,car_owners_unique_40_60_prcntl,car_owners_unique_60_80_prcntl,car_owners_unique_80_100_prcntl,carsharing_user_unique_0_20_prcntl,carsharing_user_unique_20_40_prcntl,carsharing_user_unique_40_60_prcntl,carsharing_user_unique_60_80_prcntl,carsharing_user_unique_80_100_prcntl,not_citizen_rf_unique_0_20_prcntl,not_citizen_rf_unique_20_40_prcntl,not_citizen_rf_unique_40_60_prcntl,not_citizen_rf_unique_60_80_prcntl,not_citizen_rf_unique_80_100_prcntl,healthy_food_inter_unique_0_20_prcntl,healthy_food_inter_unique_20_40_prcntl,healthy_food_inter_unique_40_60_prcntl,healthy_food_inter_unique_60_80_prcntl,healthy_food_inter_unique_80_100_prcntl,public_trnsprt_user_unique_0_20_prcntl,public_trnsprt_user_unique_20_40_prcntl,public_trnsprt_user_unique_40_60_prcntl,public_trnsprt_user_unique_60_80_prcntl,public_trnsprt_user_unique_80_100_prcntl,pharmacies_inter_unique_0_20_prcntl,pharmacies_inter_unique_20_40_prcntl,pharmacies_inter_unique_40_60_prcntl,pharmacies_inter_unique_60_80_prcntl,...,male_unique_0_20_prcntl,male_unique_20_40_prcntl,male_unique_40_60_prcntl,male_unique_60_80_prcntl,male_unique_80_100_prcntl,female_unique_0_20_prcntl,female_unique_20_40_prcntl,female_unique_40_60_prcntl,female_unique_60_80_prcntl,female_unique_80_100_prcntl,salary_1_unique_0_20_prcntl,salary_1_unique_20_40_prcntl,salary_1_unique_40_60_prcntl,salary_1_unique_60_80_prcntl,salary_1_unique_80_100_prcntl,salary_2_unique_0_20_prcntl,salary_2_unique_20_40_prcntl,salary_2_unique_40_60_prcntl,salary_2_unique_60_80_prcntl,salary_2_unique_80_100_prcntl,salary_3_unique_0_20_prcntl,salary_3_unique_20_40_prcntl,salary_3_unique_40_60_prcntl,salary_3_unique_60_80_prcntl,salary_3_unique_80_100_prcntl,salary_4_unique_0_20_prcntl,salary_4_unique_20_40_prcntl,salary_4_unique_40_60_prcntl,salary_4_unique_60_80_prcntl,salary_4_unique_80_100_prcntl,salary_5_unique_0_20_prcntl,salary_5_unique_20_40_prcntl,salary_5_unique_40_60_prcntl,salary_5_unique_60_80_prcntl,salary_5_unique_80_100_prcntl
0,8911060006bffff,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,89110600077ffff,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
2,89110600113ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
7,8911060018fffff,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
9,8911060019bffff,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0


**work with share df, with own percentiles**

In [44]:
df_hex_unique.head(3)

,h3_9,unique_visits,car_owners_unique,carsharing_user_unique,not_citizen_rf_unique,healthy_food_inter_unique,public_trnsprt_user_unique,pharmacies_inter_unique,beauty_salon_inter_unique,elite_sports_inter_unique,sprtswr_ftwr_inter_unique,sprt_good_inter_unique,trvl_rus_unique,trvl_abrd_unique,sprt_fit_inter_unique,hunting_fishing_inter_unique,fashion_inter_unique,coffee_inter_unique,med_serv_inter_unique,education_inter_unique,delivery_inter_unique,markets_inter_unique,no_children_unique,age_20_unique,age_20_30_unique,age_30_40_unique,age_40_60_unique,age_60_unique,male_unique,female_unique,salary_1_unique,salary_2_unique,salary_3_unique,salary_4_unique,salary_5_unique,car_owners_unique_share,carsharing_user_unique_share,not_citizen_rf_unique_share,healthy_food_inter_unique_share,public_trnsprt_user_unique_share,pharmacies_inter_unique_share,beauty_salon_inter_unique_share,elite_sports_inter_unique_share,sprtswr_ftwr_inter_unique_share,sprt_good_inter_unique_share,trvl_rus_unique_share,trvl_abrd_unique_share,sprt_fit_inter_unique_share,hunting_fishing_inter_unique_share,fashion_inter_unique_share,coffee_inter_unique_share,med_serv_inter_unique_share,education_inter_unique_share,delivery_inter_unique_share,markets_inter_unique_share,no_children_unique_share,age_20_unique_share,age_20_30_unique_share,age_30_40_unique_share,age_40_60_unique_share,age_60_unique_share,male_unique_share,female_unique_share,salary_1_unique_share,salary_2_unique_share,salary_3_unique_share,salary_4_unique_share,salary_5_unique_share
0,8911060006bffff,98,50,12,44,3,4,38,4,1,1,3,22,4,2,2,5,6,14,8,22,41,90,0,12,29,41,14,64,34,29,15,12,13,16,0.510204,0.122449,0.448980,0.030612,0.040816,0.387755,0.040816,0.010204,0.010204,0.030612,0.224490,0.040816,0.020408,0.020408,0.051020,0.061224,0.142857,0.081633,0.224490,0.418367,0.918367,0.000000,0.125000,0.302083,0.427083,0.145833,0.653061,0.346939,0.341176,0.176471,0.141176,0.152941,0.188235
1,89110600077ffff,99,53,27,48,1,3,42,1,3,1,10,27,8,4,5,4,13,17,15,26,48,83,1,12,26,39,12,69,30,20,13,20,18,16,0.535354,0.272727,0.484848,0.010101,0.030303,0.424242,0.010101,0.030303,0.010101,0.101010,0.272727,0.080808,0.040404,0.050505,0.040404,0.131313,0.171717,0.151515,0.262626,0.484848,0.838384,0.011111,0.133333,0.288889,0.433333,0.133333,0.696970,0.303030,0.229885,0.149425,0.229885,0.206897,0.183908
2,89110600113ffff,144,79,31,80,4,5,66,6,1,0,14,21,3,7,4,17,20,32,28,34,66,118,1,10,51,62,13,92,52,42,26,22,20,19,0.548611,0.215278,0.555556,0.027778,0.034722,0.458333,0.041667,0.006944,0.000000,0.097222,0.145833,0.020833,0.048611,0.027778,0.118056,0.138889,0.222222,0.194444,0.236111,0.458333,0.819444,0.007299,0.072993,0.372263,0.452555,0.094891,0.638889,0.361111,0.325581,0.201550,0.170543,0.155039,0.147287


In [45]:
hex_unique_share_percentiles = filter_columns_and_calculate_percentiles(df_hex_unique, ['share'], [0, 20, 40, 60, 80, 100],
                                                                      mode = 'include')
len(hex_unique_share_percentiles)

33

In [46]:
hex_unique_share_percentiles

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,car_owners_unique_share,0.109375,0.418520,0.462931,0.500000,0.541275,0.813953
1,carsharing_user_unique_share,0.027778,0.141694,0.172701,0.200000,0.238132,0.413333
2,not_citizen_rf_unique_share,0.300000,0.472727,0.500000,0.527974,0.557748,0.707317
3,healthy_food_inter_unique_share,0.000000,0.013316,0.023810,0.033333,0.047619,0.120000
4,public_trnsprt_user_unique_share,0.000000,0.025157,0.037736,0.049250,0.065114,0.140351
5,pharmacies_inter_unique_share,0.166667,0.450000,0.488189,0.521277,0.560501,0.744186
6,beauty_salon_inter_unique_share,0.000000,0.023256,0.035323,0.045455,0.060440,0.183673
7,elite_sports_inter_unique_share,0.000000,0.000000,0.008065,0.013333,0.020260,0.098592
8,sprtswr_ftwr_inter_unique_share,0.000000,0.000000,0.004311,0.010417,0.017544,0.093750
9,sprt_good_inter_unique_share,0.000000,0.090507,0.112245,0.129032,0.153846,0.298246


**add prcntl ending to mark such columns**

In [47]:
df_hex_unique_percentile_ARL_shares, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(hex_unique_share_percentiles, df_hex_unique, "h3_9")
for column in df_hex_unique_percentile_ARL_shares.columns:
    if column not in ['h3_9']:
        df_hex_unique_percentile_ARL_shares.rename(columns={column: column + '_prcntl'}, inplace=True)

In [48]:
wrong_columns_check

[]

In [49]:
df_hex_unique_percentile_ARL_shares.shape

(1335, 166)

In [50]:
df_hex_unique_percentile_ARL_shares.head(5)

,h3_9,car_owners_unique_share_0_20_prcntl,car_owners_unique_share_20_40_prcntl,car_owners_unique_share_40_60_prcntl,car_owners_unique_share_60_80_prcntl,car_owners_unique_share_80_100_prcntl,carsharing_user_unique_share_0_20_prcntl,carsharing_user_unique_share_20_40_prcntl,carsharing_user_unique_share_40_60_prcntl,carsharing_user_unique_share_60_80_prcntl,carsharing_user_unique_share_80_100_prcntl,not_citizen_rf_unique_share_0_20_prcntl,not_citizen_rf_unique_share_20_40_prcntl,not_citizen_rf_unique_share_40_60_prcntl,not_citizen_rf_unique_share_60_80_prcntl,not_citizen_rf_unique_share_80_100_prcntl,healthy_food_inter_unique_share_0_20_prcntl,healthy_food_inter_unique_share_20_40_prcntl,healthy_food_inter_unique_share_40_60_prcntl,healthy_food_inter_unique_share_60_80_prcntl,healthy_food_inter_unique_share_80_100_prcntl,public_trnsprt_user_unique_share_0_20_prcntl,public_trnsprt_user_unique_share_20_40_prcntl,public_trnsprt_user_unique_share_40_60_prcntl,public_trnsprt_user_unique_share_60_80_prcntl,public_trnsprt_user_unique_share_80_100_prcntl,pharmacies_inter_unique_share_0_20_prcntl,pharmacies_inter_unique_share_20_40_prcntl,pharmacies_inter_unique_share_40_60_prcntl,pharmacies_inter_unique_share_60_80_prcntl,pharmacies_inter_unique_share_80_100_prcntl,beauty_salon_inter_unique_share_0_20_prcntl,beauty_salon_inter_unique_share_20_40_prcntl,beauty_salon_inter_unique_share_40_60_prcntl,beauty_salon_inter_unique_share_60_80_prcntl,...,male_unique_share_0_20_prcntl,male_unique_share_20_40_prcntl,male_unique_share_40_60_prcntl,male_unique_share_60_80_prcntl,male_unique_share_80_100_prcntl,female_unique_share_0_20_prcntl,female_unique_share_20_40_prcntl,female_unique_share_40_60_prcntl,female_unique_share_60_80_prcntl,female_unique_share_80_100_prcntl,salary_1_unique_share_0_20_prcntl,salary_1_unique_share_20_40_prcntl,salary_1_unique_share_40_60_prcntl,salary_1_unique_share_60_80_prcntl,salary_1_unique_share_80_100_prcntl,salary_2_unique_share_0_20_prcntl,salary_2_unique_share_20_40_prcntl,salary_2_unique_share_40_60_prcntl,salary_2_unique_share_60_80_prcntl,salary_2_unique_share_80_100_prcntl,salary_3_unique_share_0_20_prcntl,salary_3_unique_share_20_40_prcntl,salary_3_unique_share_40_60_prcntl,salary_3_unique_share_60_80_prcntl,salary_3_unique_share_80_100_prcntl,salary_4_unique_share_0_20_prcntl,salary_4_unique_share_20_40_prcntl,salary_4_unique_share_40_60_prcntl,salary_4_unique_share_60_80_prcntl,salary_4_unique_share_80_100_prcntl,salary_5_unique_share_0_20_prcntl,salary_5_unique_share_20_40_prcntl,salary_5_unique_share_40_60_prcntl,salary_5_unique_share_60_80_prcntl,salary_5_unique_share_80_100_prcntl
0,8911060006bffff,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,89110600077ffff,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,89110600113ffff,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
7,8911060018fffff,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
9,8911060019bffff,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


**making combined df**

In [51]:
df_hex_unique_ARL_combined = pd.merge(df_hex_unique_percentile_ARL, df_hex_unique_percentile_ARL_shares,
                                how='left', left_on='h3_9', right_on='h3_9')

In [52]:
df_hex_unique_ARL_combined.shape

(1335, 336)

In [53]:
df_hex_unique_ARL_combined.head(3)

,h3_9,unique_visits_0_20_prcntl,unique_visits_20_40_prcntl,unique_visits_40_60_prcntl,unique_visits_60_80_prcntl,unique_visits_80_100_prcntl,car_owners_unique_0_20_prcntl,car_owners_unique_20_40_prcntl,car_owners_unique_40_60_prcntl,car_owners_unique_60_80_prcntl,car_owners_unique_80_100_prcntl,carsharing_user_unique_0_20_prcntl,carsharing_user_unique_20_40_prcntl,carsharing_user_unique_40_60_prcntl,carsharing_user_unique_60_80_prcntl,carsharing_user_unique_80_100_prcntl,not_citizen_rf_unique_0_20_prcntl,not_citizen_rf_unique_20_40_prcntl,not_citizen_rf_unique_40_60_prcntl,not_citizen_rf_unique_60_80_prcntl,not_citizen_rf_unique_80_100_prcntl,healthy_food_inter_unique_0_20_prcntl,healthy_food_inter_unique_20_40_prcntl,healthy_food_inter_unique_40_60_prcntl,healthy_food_inter_unique_60_80_prcntl,healthy_food_inter_unique_80_100_prcntl,public_trnsprt_user_unique_0_20_prcntl,public_trnsprt_user_unique_20_40_prcntl,public_trnsprt_user_unique_40_60_prcntl,public_trnsprt_user_unique_60_80_prcntl,public_trnsprt_user_unique_80_100_prcntl,pharmacies_inter_unique_0_20_prcntl,pharmacies_inter_unique_20_40_prcntl,pharmacies_inter_unique_40_60_prcntl,pharmacies_inter_unique_60_80_prcntl,...,male_unique_share_0_20_prcntl,male_unique_share_20_40_prcntl,male_unique_share_40_60_prcntl,male_unique_share_60_80_prcntl,male_unique_share_80_100_prcntl,female_unique_share_0_20_prcntl,female_unique_share_20_40_prcntl,female_unique_share_40_60_prcntl,female_unique_share_60_80_prcntl,female_unique_share_80_100_prcntl,salary_1_unique_share_0_20_prcntl,salary_1_unique_share_20_40_prcntl,salary_1_unique_share_40_60_prcntl,salary_1_unique_share_60_80_prcntl,salary_1_unique_share_80_100_prcntl,salary_2_unique_share_0_20_prcntl,salary_2_unique_share_20_40_prcntl,salary_2_unique_share_40_60_prcntl,salary_2_unique_share_60_80_prcntl,salary_2_unique_share_80_100_prcntl,salary_3_unique_share_0_20_prcntl,salary_3_unique_share_20_40_prcntl,salary_3_unique_share_40_60_prcntl,salary_3_unique_share_60_80_prcntl,salary_3_unique_share_80_100_prcntl,salary_4_unique_share_0_20_prcntl,salary_4_unique_share_20_40_prcntl,salary_4_unique_share_40_60_prcntl,salary_4_unique_share_60_80_prcntl,salary_4_unique_share_80_100_prcntl,salary_5_unique_share_0_20_prcntl,salary_5_unique_share_20_40_prcntl,salary_5_unique_share_40_60_prcntl,salary_5_unique_share_60_80_prcntl,salary_5_unique_share_80_100_prcntl
0,8911060006bffff,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,89110600077ffff,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,89110600113ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0


**making combined percentile df to check rangesin future**

In [54]:
hex_unique_share_percentiles.shape, hex_unique_percentiles.shape

((33, 7), (34, 7))

In [55]:
df_hex_unique_percentile_ranges = pd.concat([hex_unique_share_percentiles, hex_unique_percentiles], ignore_index=True)

In [56]:
df_hex_unique_percentile_ranges.shape

(67, 7)

In [57]:
df_hex_unique_percentile_ranges.head(10)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,car_owners_unique_share,0.109375,0.418520,0.462931,0.500000,0.541275,0.813953
1,carsharing_user_unique_share,0.027778,0.141694,0.172701,0.200000,0.238132,0.413333
2,not_citizen_rf_unique_share,0.300000,0.472727,0.500000,0.527974,0.557748,0.707317
3,healthy_food_inter_unique_share,0.000000,0.013316,0.023810,0.033333,0.047619,0.120000
4,public_trnsprt_user_unique_share,0.000000,0.025157,0.037736,0.049250,0.065114,0.140351
5,pharmacies_inter_unique_share,0.166667,0.450000,0.488189,0.521277,0.560501,0.744186
6,beauty_salon_inter_unique_share,0.000000,0.023256,0.035323,0.045455,0.060440,0.183673
7,elite_sports_inter_unique_share,0.000000,0.000000,0.008065,0.013333,0.020260,0.098592
8,sprtswr_ftwr_inter_unique_share,0.000000,0.000000,0.004311,0.010417,0.017544,0.093750
9,sprt_good_inter_unique_share,0.000000,0.090507,0.112245,0.129032,0.153846,0.298246


In [58]:
#df_hex_unique_percentile_ranges
#df_hex_unique_ARL_combined

## df_time dataset

In [59]:
df_time.head(3)

,h3_9,avg_duration_hrs,median_duration_hrs,weekday_visits,weekend_visits,Stay05_1h,Stay1_3h,Stay3_6h,Stay6_9h,Stay9_15h,Stay15_20h,Stay20_plus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,day,evening,morning,night,weekday_visits_share,weekend_visits_share,Stay05_1h_share,Stay1_3h_share,Stay3_6h_share,Stay6_9h_share,Stay9_15h_share,Stay15_20h_share,Stay20_plus_share,Friday_share,Monday_share,Saturday_share,Sunday_share,Thursday_share,Tuesday_share,Wednesday_share,day_share,evening_share,morning_share,night_share,avg_duration_hrs_per_unique,median_duration_hrs_per_unique,Stay05_1h_per_unique,Stay1_3h_per_unique,Stay3_6h_per_unique,Stay6_9h_per_unique,Stay9_15h_per_unique,Stay15_20h_per_unique,Stay20_plus_per_unique,Stay05_1h_per_unique_share_per_unique,Stay1_3h_per_unique_share_per_unique,Stay3_6h_per_unique_share_per_unique,Stay6_9h_per_unique_share_per_unique,Stay9_15h_per_unique_share_per_unique,Stay15_20h_per_unique_share_per_unique,Stay20_plus_per_unique_share_per_unique
0,8911060006bffff,4.722789,2.23,108,39,10,76,19,15,21,2,4,37,25,24,15,12,21,13,47,40,28,32,0.734694,0.265306,0.068027,0.517007,0.129252,0.102041,0.142857,0.013605,0.027211,0.251701,0.170068,0.163265,0.102041,0.081633,0.142857,0.088435,0.319728,0.272109,0.190476,0.217687,3.710017,1.70,8,57,13,9,9,1,1,0.081633,0.581633,0.132653,0.091837,0.091837,0.010204,0.010204
1,89110600077ffff,3.709160,1.81,186,64,22,141,44,17,16,9,1,36,54,31,33,34,36,26,115,54,36,45,0.744000,0.256000,0.088000,0.564000,0.176000,0.068000,0.064000,0.036000,0.004000,0.144000,0.216000,0.124000,0.132000,0.136000,0.144000,0.104000,0.460000,0.216000,0.144000,0.180000,2.982734,1.39,18,55,13,6,4,3,0,0.181818,0.555556,0.131313,0.060606,0.040404,0.030303,0.000000
2,89110600113ffff,4.214955,1.72,213,120,28,229,39,18,9,7,3,46,61,62,58,36,33,37,148,51,84,50,0.639640,0.360360,0.084084,0.687688,0.117117,0.054054,0.027027,0.021021,0.009009,0.138138,0.183183,0.186186,0.174174,0.108108,0.099099,0.111111,0.444444,0.153153,0.252252,0.150150,5.845619,1.50,20,93,14,7,5,3,2,0.138889,0.645833,0.097222,0.048611,0.034722,0.020833,0.013889


In [60]:
time_percentiles = filter_columns_and_calculate_percentiles(df_time, ['share', 'h3_9'], [0, 20, 40, 60, 80, 100],
                                                           mode = 'exclude')
time_percentiles.head(15).round(2)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,avg_duration_hrs,1.86,4.39,5.09,5.73,6.53,12.02
1,median_duration_hrs,1.31,2.05,2.41,2.72,3.05,7.99
2,weekday_visits,43.00,208.00,319.00,461.00,673.00,2310.00
3,weekend_visits,18.00,117.80,181.60,250.00,354.20,1129.00
4,Stay05_1h,0.00,16.00,28.00,43.00,67.00,259.00
5,Stay1_3h,32.00,166.00,249.60,345.00,493.00,1590.00
6,Stay3_6h,3.00,54.00,83.00,120.00,171.00,619.00
7,Stay6_9h,0.00,27.00,45.60,71.00,110.00,524.00
8,Stay9_15h,1.00,28.00,50.00,77.00,123.00,462.00
9,Stay15_20h,0.00,7.00,13.00,22.00,35.00,101.00


In [61]:
time_percentiles.shape

(31, 7)

**add prcntl ending to mark such columns**

In [62]:
df_time_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(time_percentiles, df_time, "h3_9")
for column in df_time_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_time_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [63]:
wrong_columns_check

[]

In [64]:
df_time_percentile_ARL.shape

(1335, 156)

In [65]:
df_time_percentile_ARL.head(5)

,h3_9,avg_duration_hrs_0_20_prcntl,avg_duration_hrs_20_40_prcntl,avg_duration_hrs_40_60_prcntl,avg_duration_hrs_60_80_prcntl,avg_duration_hrs_80_100_prcntl,median_duration_hrs_0_20_prcntl,median_duration_hrs_20_40_prcntl,median_duration_hrs_40_60_prcntl,median_duration_hrs_60_80_prcntl,median_duration_hrs_80_100_prcntl,weekday_visits_0_20_prcntl,weekday_visits_20_40_prcntl,weekday_visits_40_60_prcntl,weekday_visits_60_80_prcntl,weekday_visits_80_100_prcntl,weekend_visits_0_20_prcntl,weekend_visits_20_40_prcntl,weekend_visits_40_60_prcntl,weekend_visits_60_80_prcntl,weekend_visits_80_100_prcntl,Stay05_1h_0_20_prcntl,Stay05_1h_20_40_prcntl,Stay05_1h_40_60_prcntl,Stay05_1h_60_80_prcntl,Stay05_1h_80_100_prcntl,Stay1_3h_0_20_prcntl,Stay1_3h_20_40_prcntl,Stay1_3h_40_60_prcntl,Stay1_3h_60_80_prcntl,Stay1_3h_80_100_prcntl,Stay3_6h_0_20_prcntl,Stay3_6h_20_40_prcntl,Stay3_6h_40_60_prcntl,Stay3_6h_60_80_prcntl,...,Stay05_1h_per_unique_0_20_prcntl,Stay05_1h_per_unique_20_40_prcntl,Stay05_1h_per_unique_40_60_prcntl,Stay05_1h_per_unique_60_80_prcntl,Stay05_1h_per_unique_80_100_prcntl,Stay1_3h_per_unique_0_20_prcntl,Stay1_3h_per_unique_20_40_prcntl,Stay1_3h_per_unique_40_60_prcntl,Stay1_3h_per_unique_60_80_prcntl,Stay1_3h_per_unique_80_100_prcntl,Stay3_6h_per_unique_0_20_prcntl,Stay3_6h_per_unique_20_40_prcntl,Stay3_6h_per_unique_40_60_prcntl,Stay3_6h_per_unique_60_80_prcntl,Stay3_6h_per_unique_80_100_prcntl,Stay6_9h_per_unique_0_20_prcntl,Stay6_9h_per_unique_20_40_prcntl,Stay6_9h_per_unique_40_60_prcntl,Stay6_9h_per_unique_60_80_prcntl,Stay6_9h_per_unique_80_100_prcntl,Stay9_15h_per_unique_0_20_prcntl,Stay9_15h_per_unique_20_40_prcntl,Stay9_15h_per_unique_40_60_prcntl,Stay9_15h_per_unique_60_80_prcntl,Stay9_15h_per_unique_80_100_prcntl,Stay15_20h_per_unique_0_20_prcntl,Stay15_20h_per_unique_20_40_prcntl,Stay15_20h_per_unique_40_60_prcntl,Stay15_20h_per_unique_60_80_prcntl,Stay15_20h_per_unique_80_100_prcntl,Stay20_plus_per_unique_0_20_prcntl,Stay20_plus_per_unique_20_40_prcntl,Stay20_plus_per_unique_40_60_prcntl,Stay20_plus_per_unique_60_80_prcntl,Stay20_plus_per_unique_80_100_prcntl
0,8911060006bffff,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0
1,89110600077ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,89110600113ffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
7,8911060018fffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0
9,8911060019bffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0


**work with share df, also with percentiles**

In [66]:
df_time.head(5)

,h3_9,avg_duration_hrs,median_duration_hrs,weekday_visits,weekend_visits,Stay05_1h,Stay1_3h,Stay3_6h,Stay6_9h,Stay9_15h,Stay15_20h,Stay20_plus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,day,evening,morning,night,weekday_visits_share,weekend_visits_share,Stay05_1h_share,Stay1_3h_share,Stay3_6h_share,Stay6_9h_share,Stay9_15h_share,Stay15_20h_share,Stay20_plus_share,Friday_share,Monday_share,Saturday_share,Sunday_share,Thursday_share,Tuesday_share,Wednesday_share,day_share,evening_share,morning_share,night_share,avg_duration_hrs_per_unique,median_duration_hrs_per_unique,Stay05_1h_per_unique,Stay1_3h_per_unique,Stay3_6h_per_unique,Stay6_9h_per_unique,Stay9_15h_per_unique,Stay15_20h_per_unique,Stay20_plus_per_unique,Stay05_1h_per_unique_share_per_unique,Stay1_3h_per_unique_share_per_unique,Stay3_6h_per_unique_share_per_unique,Stay6_9h_per_unique_share_per_unique,Stay9_15h_per_unique_share_per_unique,Stay15_20h_per_unique_share_per_unique,Stay20_plus_per_unique_share_per_unique
0,8911060006bffff,4.722789,2.23,108,39,10,76,19,15,21,2,4,37,25,24,15,12,21,13,47,40,28,32,0.734694,0.265306,0.068027,0.517007,0.129252,0.102041,0.142857,0.013605,0.027211,0.251701,0.170068,0.163265,0.102041,0.081633,0.142857,0.088435,0.319728,0.272109,0.190476,0.217687,3.710017,1.70,8,57,13,9,9,1,1,0.081633,0.581633,0.132653,0.091837,0.091837,0.010204,0.010204
1,89110600077ffff,3.709160,1.81,186,64,22,141,44,17,16,9,1,36,54,31,33,34,36,26,115,54,36,45,0.744000,0.256000,0.088000,0.564000,0.176000,0.068000,0.064000,0.036000,0.004000,0.144000,0.216000,0.124000,0.132000,0.136000,0.144000,0.104000,0.460000,0.216000,0.144000,0.180000,2.982734,1.39,18,55,13,6,4,3,0,0.181818,0.555556,0.131313,0.060606,0.040404,0.030303,0.000000
2,89110600113ffff,4.214955,1.72,213,120,28,229,39,18,9,7,3,46,61,62,58,36,33,37,148,51,84,50,0.639640,0.360360,0.084084,0.687688,0.117117,0.054054,0.027027,0.021021,0.009009,0.138138,0.183183,0.186186,0.174174,0.108108,0.099099,0.111111,0.444444,0.153153,0.252252,0.150150,5.845619,1.50,20,93,14,7,5,3,2,0.138889,0.645833,0.097222,0.048611,0.034722,0.020833,0.013889
7,8911060018fffff,3.216036,1.85,288,151,44,273,62,29,23,4,4,45,90,67,84,48,42,63,198,84,78,79,0.656036,0.343964,0.100228,0.621868,0.141230,0.066059,0.052392,0.009112,0.009112,0.102506,0.205011,0.152620,0.191344,0.109339,0.095672,0.143508,0.451025,0.191344,0.177677,0.179954,3.199664,1.62,20,91,28,8,8,3,1,0.125786,0.572327,0.176101,0.050314,0.050314,0.018868,0.006289
9,8911060019bffff,3.214016,1.72,143,106,10,182,28,13,9,3,4,32,40,44,62,13,29,29,125,26,65,33,0.574297,0.425703,0.040161,0.730924,0.112450,0.052209,0.036145,0.012048,0.016064,0.128514,0.160643,0.176707,0.248996,0.052209,0.116466,0.116466,0.502008,0.104418,0.261044,0.132530,3.847617,1.69,2,56,8,5,3,1,3,0.025641,0.717949,0.102564,0.064103,0.038462,0.012821,0.038462


In [67]:
time_share_percentiles = filter_columns_and_calculate_percentiles(df_time, ['share'], [0, 20, 40, 60, 80, 100],
                                                                      mode = 'include')
len(time_share_percentiles)

27

In [68]:
time_share_percentiles

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,weekday_visits_share,0.329004,0.623243,0.642992,0.654862,0.669048,0.810526
1,weekend_visits_share,0.189474,0.330952,0.345138,0.357008,0.376757,0.670996
2,Stay05_1h_share,0.000000,0.036545,0.051308,0.064843,0.082786,0.193548
3,Stay1_3h_share,0.274854,0.439047,0.472037,0.513927,0.563195,0.844262
4,Stay3_6h_share,0.030303,0.146705,0.161178,0.173155,0.185286,0.314356
5,Stay6_9h_share,0.000000,0.076322,0.092518,0.105348,0.119801,0.209150
6,Stay9_15h_share,0.008130,0.074807,0.099242,0.116833,0.137646,0.256959
7,Stay15_20h_share,0.000000,0.016788,0.025622,0.033474,0.044071,0.137427
8,Stay20_plus_share,0.000000,0.015996,0.025893,0.035439,0.049261,0.159509
9,Friday_share,0.004950,0.115745,0.137734,0.151515,0.165194,0.331818


**add prcntl ending to mark such columns**

In [69]:
df_time_percentile_ARL_shares, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(time_share_percentiles, df_time, "h3_9")
for column in df_time_percentile_ARL_shares.columns:
    if column not in ['h3_9']:
        df_time_percentile_ARL_shares.rename(columns={column: column + '_prcntl'}, inplace=True)

In [70]:
wrong_columns_check

[]

In [71]:
df_time_percentile_ARL_shares.shape

(1335, 136)

In [72]:
df_time_percentile_ARL_shares.head(5)

,h3_9,weekday_visits_share_0_20_prcntl,weekday_visits_share_20_40_prcntl,weekday_visits_share_40_60_prcntl,weekday_visits_share_60_80_prcntl,weekday_visits_share_80_100_prcntl,weekend_visits_share_0_20_prcntl,weekend_visits_share_20_40_prcntl,weekend_visits_share_40_60_prcntl,weekend_visits_share_60_80_prcntl,weekend_visits_share_80_100_prcntl,Stay05_1h_share_0_20_prcntl,Stay05_1h_share_20_40_prcntl,Stay05_1h_share_40_60_prcntl,Stay05_1h_share_60_80_prcntl,Stay05_1h_share_80_100_prcntl,Stay1_3h_share_0_20_prcntl,Stay1_3h_share_20_40_prcntl,Stay1_3h_share_40_60_prcntl,Stay1_3h_share_60_80_prcntl,Stay1_3h_share_80_100_prcntl,Stay3_6h_share_0_20_prcntl,Stay3_6h_share_20_40_prcntl,Stay3_6h_share_40_60_prcntl,Stay3_6h_share_60_80_prcntl,Stay3_6h_share_80_100_prcntl,Stay6_9h_share_0_20_prcntl,Stay6_9h_share_20_40_prcntl,Stay6_9h_share_40_60_prcntl,Stay6_9h_share_60_80_prcntl,Stay6_9h_share_80_100_prcntl,Stay9_15h_share_0_20_prcntl,Stay9_15h_share_20_40_prcntl,Stay9_15h_share_40_60_prcntl,Stay9_15h_share_60_80_prcntl,...,Stay05_1h_per_unique_share_per_unique_0_20_prcntl,Stay05_1h_per_unique_share_per_unique_20_40_prcntl,Stay05_1h_per_unique_share_per_unique_40_60_prcntl,Stay05_1h_per_unique_share_per_unique_60_80_prcntl,Stay05_1h_per_unique_share_per_unique_80_100_prcntl,Stay1_3h_per_unique_share_per_unique_0_20_prcntl,Stay1_3h_per_unique_share_per_unique_20_40_prcntl,Stay1_3h_per_unique_share_per_unique_40_60_prcntl,Stay1_3h_per_unique_share_per_unique_60_80_prcntl,Stay1_3h_per_unique_share_per_unique_80_100_prcntl,Stay3_6h_per_unique_share_per_unique_0_20_prcntl,Stay3_6h_per_unique_share_per_unique_20_40_prcntl,Stay3_6h_per_unique_share_per_unique_40_60_prcntl,Stay3_6h_per_unique_share_per_unique_60_80_prcntl,Stay3_6h_per_unique_share_per_unique_80_100_prcntl,Stay6_9h_per_unique_share_per_unique_0_20_prcntl,Stay6_9h_per_unique_share_per_unique_20_40_prcntl,Stay6_9h_per_unique_share_per_unique_40_60_prcntl,Stay6_9h_per_unique_share_per_unique_60_80_prcntl,Stay6_9h_per_unique_share_per_unique_80_100_prcntl,Stay9_15h_per_unique_share_per_unique_0_20_prcntl,Stay9_15h_per_unique_share_per_unique_20_40_prcntl,Stay9_15h_per_unique_share_per_unique_40_60_prcntl,Stay9_15h_per_unique_share_per_unique_60_80_prcntl,Stay9_15h_per_unique_share_per_unique_80_100_prcntl,Stay15_20h_per_unique_share_per_unique_0_20_prcntl,Stay15_20h_per_unique_share_per_unique_20_40_prcntl,Stay15_20h_per_unique_share_per_unique_40_60_prcntl,Stay15_20h_per_unique_share_per_unique_60_80_prcntl,Stay15_20h_per_unique_share_per_unique_80_100_prcntl,Stay20_plus_per_unique_share_per_unique_0_20_prcntl,Stay20_plus_per_unique_share_per_unique_20_40_prcntl,Stay20_plus_per_unique_share_per_unique_40_60_prcntl,Stay20_plus_per_unique_share_per_unique_60_80_prcntl,Stay20_plus_per_unique_share_per_unique_80_100_prcntl
0,8911060006bffff,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0
1,89110600077ffff,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
2,89110600113ffff,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
7,8911060018fffff,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
9,8911060019bffff,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0


**making combined df**

In [73]:
df_time_ARL_combined = pd.merge(df_time_percentile_ARL, df_time_percentile_ARL_shares,
                                how='left', left_on='h3_9', right_on='h3_9')

In [74]:
df_time_ARL_combined.shape

(1335, 291)

In [75]:
df_time_ARL_combined.head(3)

,h3_9,avg_duration_hrs_0_20_prcntl,avg_duration_hrs_20_40_prcntl,avg_duration_hrs_40_60_prcntl,avg_duration_hrs_60_80_prcntl,avg_duration_hrs_80_100_prcntl,median_duration_hrs_0_20_prcntl,median_duration_hrs_20_40_prcntl,median_duration_hrs_40_60_prcntl,median_duration_hrs_60_80_prcntl,median_duration_hrs_80_100_prcntl,weekday_visits_0_20_prcntl,weekday_visits_20_40_prcntl,weekday_visits_40_60_prcntl,weekday_visits_60_80_prcntl,weekday_visits_80_100_prcntl,weekend_visits_0_20_prcntl,weekend_visits_20_40_prcntl,weekend_visits_40_60_prcntl,weekend_visits_60_80_prcntl,weekend_visits_80_100_prcntl,Stay05_1h_0_20_prcntl,Stay05_1h_20_40_prcntl,Stay05_1h_40_60_prcntl,Stay05_1h_60_80_prcntl,Stay05_1h_80_100_prcntl,Stay1_3h_0_20_prcntl,Stay1_3h_20_40_prcntl,Stay1_3h_40_60_prcntl,Stay1_3h_60_80_prcntl,Stay1_3h_80_100_prcntl,Stay3_6h_0_20_prcntl,Stay3_6h_20_40_prcntl,Stay3_6h_40_60_prcntl,Stay3_6h_60_80_prcntl,...,Stay05_1h_per_unique_share_per_unique_0_20_prcntl,Stay05_1h_per_unique_share_per_unique_20_40_prcntl,Stay05_1h_per_unique_share_per_unique_40_60_prcntl,Stay05_1h_per_unique_share_per_unique_60_80_prcntl,Stay05_1h_per_unique_share_per_unique_80_100_prcntl,Stay1_3h_per_unique_share_per_unique_0_20_prcntl,Stay1_3h_per_unique_share_per_unique_20_40_prcntl,Stay1_3h_per_unique_share_per_unique_40_60_prcntl,Stay1_3h_per_unique_share_per_unique_60_80_prcntl,Stay1_3h_per_unique_share_per_unique_80_100_prcntl,Stay3_6h_per_unique_share_per_unique_0_20_prcntl,Stay3_6h_per_unique_share_per_unique_20_40_prcntl,Stay3_6h_per_unique_share_per_unique_40_60_prcntl,Stay3_6h_per_unique_share_per_unique_60_80_prcntl,Stay3_6h_per_unique_share_per_unique_80_100_prcntl,Stay6_9h_per_unique_share_per_unique_0_20_prcntl,Stay6_9h_per_unique_share_per_unique_20_40_prcntl,Stay6_9h_per_unique_share_per_unique_40_60_prcntl,Stay6_9h_per_unique_share_per_unique_60_80_prcntl,Stay6_9h_per_unique_share_per_unique_80_100_prcntl,Stay9_15h_per_unique_share_per_unique_0_20_prcntl,Stay9_15h_per_unique_share_per_unique_20_40_prcntl,Stay9_15h_per_unique_share_per_unique_40_60_prcntl,Stay9_15h_per_unique_share_per_unique_60_80_prcntl,Stay9_15h_per_unique_share_per_unique_80_100_prcntl,Stay15_20h_per_unique_share_per_unique_0_20_prcntl,Stay15_20h_per_unique_share_per_unique_20_40_prcntl,Stay15_20h_per_unique_share_per_unique_40_60_prcntl,Stay15_20h_per_unique_share_per_unique_60_80_prcntl,Stay15_20h_per_unique_share_per_unique_80_100_prcntl,Stay20_plus_per_unique_share_per_unique_0_20_prcntl,Stay20_plus_per_unique_share_per_unique_20_40_prcntl,Stay20_plus_per_unique_share_per_unique_40_60_prcntl,Stay20_plus_per_unique_share_per_unique_60_80_prcntl,Stay20_plus_per_unique_share_per_unique_80_100_prcntl
0,8911060006bffff,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0
1,89110600077ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
2,89110600113ffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0


**making combined percentile df to check rangesin future**

In [76]:
time_share_percentiles.shape, time_percentiles.shape

((27, 7), (31, 7))

In [77]:
df_time_percentile_ranges = pd.concat([time_share_percentiles, time_percentiles], ignore_index=True)

In [78]:
df_time_percentile_ranges.shape

(58, 7)

In [79]:
df_time_percentile_ranges.head(10)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,weekday_visits_share,0.329004,0.623243,0.642992,0.654862,0.669048,0.810526
1,weekend_visits_share,0.189474,0.330952,0.345138,0.357008,0.376757,0.670996
2,Stay05_1h_share,0.000000,0.036545,0.051308,0.064843,0.082786,0.193548
3,Stay1_3h_share,0.274854,0.439047,0.472037,0.513927,0.563195,0.844262
4,Stay3_6h_share,0.030303,0.146705,0.161178,0.173155,0.185286,0.314356
5,Stay6_9h_share,0.000000,0.076322,0.092518,0.105348,0.119801,0.209150
6,Stay9_15h_share,0.008130,0.074807,0.099242,0.116833,0.137646,0.256959
7,Stay15_20h_share,0.000000,0.016788,0.025622,0.033474,0.044071,0.137427
8,Stay20_plus_share,0.000000,0.015996,0.025893,0.035439,0.049261,0.159509
9,Friday_share,0.004950,0.115745,0.137734,0.151515,0.165194,0.331818


In [80]:
#df_time_percentile_ranges
#df_time_ARL_combined

## df_geo dataset

**waiting for data from Sasha. Now, I make it without OSM data, because of it strange distributions**

**OHE for distict name**

In [167]:
df_geo = pd.get_dummies(df_geo, columns = ['name'])

In [168]:
adj_col_names = []
for name in df_geo.columns:
    adj_col_names.append(name.replace('name_',''))

df_geo.columns = adj_col_names

In [169]:
df_geo.shape

(1335, 110)

In [170]:
rename_dict_geo = {
    'Адмиралтейский район': 'admiralteysky_district',
    'Василеостровский район': 'vasileostrovsky_district',
    'Выборгский район': 'vyborgsky_district',
    'Калининский район': 'kalininsky_district',
    'Кировский район': 'kirovsky_district',
    'Красногвардейский район': 'krasnogvardeysky_district',
    'Красносельский район': 'krasnoselsky_district',
    'Московский район': 'moskovsky_district',
    'Невский район': 'nevsky_district',
    'Петроградский район': 'petrogradsky_district',
    'Приморский район': 'primorsky_district',
    'Пушкинский район': 'pushkinsky_district',
    'Фрунзенский район': 'frunzensky_district',
    'Центральный район': 'tsentralny_district'
}

# Rename columns using the dictionary
df_geo.rename(columns=rename_dict_geo, inplace=True)

In [176]:
df_geo.filter(regex = 'district').columns

Index(['admiralteysky_district', 'vasileostrovsky_district',
       'vyborgsky_district', 'kalininsky_district', 'kirovsky_district',
       'krasnogvardeysky_district', 'krasnoselsky_district',
       'moskovsky_district', 'nevsky_district', 'petrogradsky_district',
       'primorsky_district', 'pushkinsky_district', 'frunzensky_district',
       'tsentralny_district'],
      dtype='object')

In [172]:
df_geo.head(3)

,h3_9,cian_total_listing_count_including_hex_3hn,cian_total_area_including_hex_3hn,cian_median_area_including_hex_3hn,cian_avg_area_including_hex_3hn,cian_median_price_per_m2_including_hex_3hn,cian_avg_price_per_m2_including_hex_3hn,spb_trans_amount_unqiue_vehicles,spb_trans_unique_vehicle_stops,spb_trans_unique_stops,spb_trans_bus_stop_counter,spb_trans_tram_stop_counter,spb_trans_trolleybus_stop_counter,total_fit_clubs,total_fit_clubs_neighbour_2_hn,metro_count,total_metro_neighbour_5_hn,population_sum,total_population_neighbour_2_hn,category_auto_count_2_hn,category_security_count_2_hn,category_business_count_2_hn,category_government_count_2_hn,category_pets_count_2_hn,category_home_and_interior_count_2_hn,category_food_and_drinks_count_2_hn,category_computers_and_internet_count_2_hn,category_beauty_and_health_count_2_hn,category_culture_and_art_count_2_hn,category_stores_count_2_hn,category_medicine_and_pharmacy_count_2_hn,category_hospitals_count_2_hn,category_science_and_education_count_2_hn,category_kindergartens_count_2_hn,category_universities_count_2_hn,...,subcategory_dormitories_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_entertainment_centers_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_hotels_count_2_hn,subcategory_parking_lots_and_garages_count_2_hn,subcategory_free_parking_count_2_hn,subcategory_parking_count_2_hn,subcategory_paid_parking_count_2_hn,subcategory_underground_parking_count_2_hn,subcategory_polyclinics_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_general_education_schools_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,subcategory_industrial_enterprises_count_2_hn,admiralteysky_district,vasileostrovsky_district,vyborgsky_district,kalininsky_district,kirovsky_district,krasnogvardeysky_district,krasnoselsky_district,moskovsky_district,nevsky_district,petrogradsky_district,primorsky_district,pushkinsky_district,frunzensky_district,tsentralny_district
0,8911060006bffff,8.0,12062.9,1263.33,1287.046,1000.0,937.210,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.0,15.0,11.0,2.0,11.0,17.0,43.0,8.0,0.0,1.0,30.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,2.0,6.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,89110600077ffff,136.0,237533.7,951.95,1292.975,750.0,895.748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,12.0,7.0,1.0,2.0,0.0,32.0,12.0,0.0,1.0,10.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,89110600113ffff,10.0,16975.0,746.00,944.250,980.0,1390.000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,239.0,41.0,53.0,0.0,3.0,31.0,156.0,40.0,95.0,17.0,64.0,21.0,2.0,5.0,0.0,0.0,...,0.0,0.0,0.0,3.0,4.0,0.0,6.0,6.0,22.0,15.0,22.0,14.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


**based on the geo_percentiles I am going to remain almost all columns with count in the final df (or I can simply drop it because it looks like dust), so I rename non binary count columns to counter so it will be easy to code. So, now all count columns are binary, counter - no**

In [173]:
#df_geo.rename(columns={'amenity_count': 'amenity_counter_osm', 'building_count': 'building_counter_osm'}, inplace=True)

In [174]:
pd.set_option('display.max_rows', 150)
geo_percentiles = filter_columns_and_calculate_percentiles(df_geo, ['h3_9', 'population_sum'], [0, 20, 40, 60, 80, 100])
geo_percentiles.head(150)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,cian_total_listing_count_including_hex_3hn,0.0,8.000000,16.000000,41.000000,119.000000,470.000000
1,cian_total_area_including_hex_3hn,0.0,1366.860000,4163.880000,17833.510000,49882.480000,288020.970000
2,cian_median_area_including_hex_3hn,0.0,83.100000,115.782000,147.000000,208.720000,2415.000000
3,cian_avg_area_including_hex_3hn,0.0,134.112500,202.838417,285.985079,429.919889,3805.556667
4,cian_median_price_per_m2_including_hex_3hn,0.0,1105.118000,1312.286000,1500.000000,1799.500000,3846.150000
5,cian_avg_price_per_m2_including_hex_3hn,0.0,1232.588429,1475.732857,1736.273423,2090.340833,76459.370909
6,spb_trans_amount_unqiue_vehicles,0.0,0.000000,0.000000,1.000000,2.000000,3.000000
7,spb_trans_unique_vehicle_stops,0.0,0.000000,0.000000,1.000000,2.000000,8.000000
8,spb_trans_unique_stops,0.0,0.000000,0.000000,1.000000,2.000000,7.000000
9,spb_trans_bus_stop_counter,0.0,0.000000,0.000000,1.000000,1.000000,4.000000


In [175]:
geo_percentiles.shape

(108, 7)

In [182]:
def classify_columns(df):
    all_diff = []
    at_least_two_same = []

    for index, row in df.iterrows():
        percentiles = [row['0_percentile'], row['20_percentile'], row['40_percentile'], row['60_percentile'], row['80_percentile'], row['100_percentile']]
        
        if len(set(percentiles)) == len(percentiles):
            all_diff.append(row['column_name'])
        elif len(set(percentiles)) <= 4:
            at_least_two_same.append(row['column_name'])
    
    return all_diff, at_least_two_same

all_diff_columns, at_least_two_same_columns = classify_columns(geo_percentiles)

In [183]:
at_least_two_same_columns

['spb_trans_amount_unqiue_vehicles',
 'spb_trans_unique_vehicle_stops',
 'spb_trans_unique_stops',
 'spb_trans_bus_stop_counter',
 'spb_trans_tram_stop_counter',
 'spb_trans_trolleybus_stop_counter',
 'total_fit_clubs',
 'metro_count',
 'total_metro_neighbour_5_hn',
 'category_universities_count_2_hn',
 'category_zoos_count_2_hn',
 'category_railway_stations_count_2_hn',
 'category_bus_stations_count_2_hn',
 'category_airports_count_2_hn',
 'category_gymnasiums_count_2_hn',
 'category_colleges_count_2_hn',
 'category_lyceums_count_2_hn',
 'category_planetariums_count_2_hn',
 'category_amusement_parks_count_2_hn',
 'subcategory_universities_count_2_hn',
 'subcategory_zoos_count_2_hn',
 'subcategory_dolphinariums_and_aquariums_count_2_hn',
 'subcategory_railway_stations_count_2_hn',
 'subcategory_bus_stations_count_2_hn',
 'subcategory_airports_count_2_hn',
 'subcategory_gymnasiums_count_2_hn',
 'subcategory_colleges_count_2_hn',
 'subcategory_lyceums_count_2_hn',
 'subcategory_planetari

## Make Binary df for binary columns

In [184]:
def create_binary_dataframe(df, columns_to_convert, id_column):
    binary_df = df.copy()
    
    for column in columns_to_convert:
        binary_df[column] = binary_df[column].apply(lambda x: 1 if x > 1 else 0)
    columns_to_keep = [id_column]
    
    final_df = pd.concat([binary_df[columns_to_convert], df[columns_to_keep]], axis=1)
    
    return final_df

In [185]:
at_least_two_same_columns.insert(0, 'h3_9')

In [186]:
df_geo[at_least_two_same_columns]

,h3_9,spb_trans_amount_unqiue_vehicles,spb_trans_unique_vehicle_stops,spb_trans_unique_stops,spb_trans_bus_stop_counter,spb_trans_tram_stop_counter,spb_trans_trolleybus_stop_counter,total_fit_clubs,metro_count,total_metro_neighbour_5_hn,category_universities_count_2_hn,category_zoos_count_2_hn,category_railway_stations_count_2_hn,category_bus_stations_count_2_hn,category_airports_count_2_hn,category_gymnasiums_count_2_hn,category_colleges_count_2_hn,category_lyceums_count_2_hn,category_planetariums_count_2_hn,category_amusement_parks_count_2_hn,subcategory_universities_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,admiralteysky_district,vasileostrovsky_district,vyborgsky_district,kalininsky_district,kirovsky_district,krasnogvardeysky_district,krasnoselsky_district,moskovsky_district,nevsky_district,petrogradsky_district,primorsky_district,pushkinsky_district,frunzensky_district,tsentralny_district
0,8911060006bffff,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,89110600077ffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,89110600113ffff,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,8911060018fffff,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,8911060019bffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865,891106ab6b7ffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3866,891106ab6bbffff,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3869,891106ab6cbffff,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3871,891106ab6d7ffff,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,12.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [187]:
all_diff_columns, at_least_two_same_columns = classify_columns(geo_percentiles)

In [188]:
df_geo_binary_part = create_binary_dataframe(df_geo, at_least_two_same_columns, 'h3_9')
df_geo_binary_part.shape

(1335, 45)

In [189]:
df_geo_binary_part.head(3)

,spb_trans_amount_unqiue_vehicles,spb_trans_unique_vehicle_stops,spb_trans_unique_stops,spb_trans_bus_stop_counter,spb_trans_tram_stop_counter,spb_trans_trolleybus_stop_counter,total_fit_clubs,metro_count,total_metro_neighbour_5_hn,category_universities_count_2_hn,category_zoos_count_2_hn,category_railway_stations_count_2_hn,category_bus_stations_count_2_hn,category_airports_count_2_hn,category_gymnasiums_count_2_hn,category_colleges_count_2_hn,category_lyceums_count_2_hn,category_planetariums_count_2_hn,category_amusement_parks_count_2_hn,subcategory_universities_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,admiralteysky_district,vasileostrovsky_district,vyborgsky_district,kalininsky_district,kirovsky_district,krasnogvardeysky_district,krasnoselsky_district,moskovsky_district,nevsky_district,petrogradsky_district,primorsky_district,pushkinsky_district,frunzensky_district,tsentralny_district,h3_9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8911060006bffff
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89110600077ffff
2,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89110600113ffff


In [ ]:
#df_geo_binary_part

## Continue with Geo features for percentile

In [190]:
pd.set_option('display.max_rows', 150)
at_least_two_same_columns += ['h3_9', 'population_sum']
geo_percentiles = filter_columns_and_calculate_percentiles(df_geo, at_least_two_same_columns, [0, 20, 40, 60, 80, 100])
geo_percentiles.head(150)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,cian_total_listing_count_including_hex_3hn,0.0,8.000000,16.000000,41.000000,119.000000,470.000000
1,cian_total_area_including_hex_3hn,0.0,1366.860000,4163.880000,17833.510000,49882.480000,288020.970000
2,cian_median_area_including_hex_3hn,0.0,83.100000,115.782000,147.000000,208.720000,2415.000000
3,cian_avg_area_including_hex_3hn,0.0,134.112500,202.838417,285.985079,429.919889,3805.556667
4,cian_median_price_per_m2_including_hex_3hn,0.0,1105.118000,1312.286000,1500.000000,1799.500000,3846.150000
5,cian_avg_price_per_m2_including_hex_3hn,0.0,1232.588429,1475.732857,1736.273423,2090.340833,76459.370909
6,total_population_neighbour_2_hn,0.0,12135.000000,20724.000000,27553.200000,37745.400000,82353.000000
7,category_auto_count_2_hn,0.0,73.800000,110.000000,145.000000,194.000000,630.000000
8,category_security_count_2_hn,0.0,17.000000,27.000000,43.000000,65.000000,228.000000
9,category_business_count_2_hn,0.0,46.800000,74.600000,112.000000,175.000000,511.000000


**add prcntl ending to mark such columns**

In [191]:
all_diff_columns.insert(0, 'h3_9')

In [194]:
df_geo_percentile_part = df_geo[all_diff_columns].copy()
df_geo_percentile_part.shape

(1335, 60)

In [195]:
df_geo_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(geo_percentiles, df_geo_percentile_part, "h3_9")
for column in df_geo_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_geo_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [196]:
wrong_columns_check

[]

In [197]:
df_geo_percentile_ARL.shape

(1335, 291)

In [198]:
df_geo_percentile_ARL.head(5)

,h3_9,cian_total_listing_count_including_hex_3hn_0_20_prcntl,cian_total_listing_count_including_hex_3hn_20_40_prcntl,cian_total_listing_count_including_hex_3hn_40_60_prcntl,cian_total_listing_count_including_hex_3hn_60_80_prcntl,cian_total_listing_count_including_hex_3hn_80_100_prcntl,cian_total_area_including_hex_3hn_0_20_prcntl,cian_total_area_including_hex_3hn_20_40_prcntl,cian_total_area_including_hex_3hn_40_60_prcntl,cian_total_area_including_hex_3hn_60_80_prcntl,cian_total_area_including_hex_3hn_80_100_prcntl,cian_median_area_including_hex_3hn_0_20_prcntl,cian_median_area_including_hex_3hn_20_40_prcntl,cian_median_area_including_hex_3hn_40_60_prcntl,cian_median_area_including_hex_3hn_60_80_prcntl,cian_median_area_including_hex_3hn_80_100_prcntl,cian_avg_area_including_hex_3hn_0_20_prcntl,cian_avg_area_including_hex_3hn_20_40_prcntl,cian_avg_area_including_hex_3hn_40_60_prcntl,cian_avg_area_including_hex_3hn_60_80_prcntl,cian_avg_area_including_hex_3hn_80_100_prcntl,cian_median_price_per_m2_including_hex_3hn_0_20_prcntl,cian_median_price_per_m2_including_hex_3hn_20_40_prcntl,cian_median_price_per_m2_including_hex_3hn_40_60_prcntl,cian_median_price_per_m2_including_hex_3hn_60_80_prcntl,cian_median_price_per_m2_including_hex_3hn_80_100_prcntl,cian_avg_price_per_m2_including_hex_3hn_0_20_prcntl,cian_avg_price_per_m2_including_hex_3hn_20_40_prcntl,cian_avg_price_per_m2_including_hex_3hn_40_60_prcntl,cian_avg_price_per_m2_including_hex_3hn_60_80_prcntl,cian_avg_price_per_m2_including_hex_3hn_80_100_prcntl,total_population_neighbour_2_hn_0_20_prcntl,total_population_neighbour_2_hn_20_40_prcntl,total_population_neighbour_2_hn_40_60_prcntl,total_population_neighbour_2_hn_60_80_prcntl,...,subcategory_free_parking_count_2_hn_0_20_prcntl,subcategory_free_parking_count_2_hn_20_40_prcntl,subcategory_free_parking_count_2_hn_40_60_prcntl,subcategory_free_parking_count_2_hn_60_80_prcntl,subcategory_free_parking_count_2_hn_80_100_prcntl,subcategory_parking_count_2_hn_0_20_prcntl,subcategory_parking_count_2_hn_20_40_prcntl,subcategory_parking_count_2_hn_40_60_prcntl,subcategory_parking_count_2_hn_60_80_prcntl,subcategory_parking_count_2_hn_80_100_prcntl,subcategory_paid_parking_count_2_hn_0_20_prcntl,subcategory_paid_parking_count_2_hn_20_40_prcntl,subcategory_paid_parking_count_2_hn_40_60_prcntl,subcategory_paid_parking_count_2_hn_60_80_prcntl,subcategory_paid_parking_count_2_hn_80_100_prcntl,subcategory_underground_parking_count_2_hn_0_20_prcntl,subcategory_underground_parking_count_2_hn_20_40_prcntl,subcategory_underground_parking_count_2_hn_40_60_prcntl,subcategory_underground_parking_count_2_hn_60_80_prcntl,subcategory_underground_parking_count_2_hn_80_100_prcntl,subcategory_polyclinics_count_2_hn_0_20_prcntl,subcategory_polyclinics_count_2_hn_20_40_prcntl,subcategory_polyclinics_count_2_hn_40_60_prcntl,subcategory_polyclinics_count_2_hn_60_80_prcntl,subcategory_polyclinics_count_2_hn_80_100_prcntl,subcategory_general_education_schools_count_2_hn_0_20_prcntl,subcategory_general_education_schools_count_2_hn_20_40_prcntl,subcategory_general_education_schools_count_2_hn_40_60_prcntl,subcategory_general_education_schools_count_2_hn_60_80_prcntl,subcategory_general_education_schools_count_2_hn_80_100_prcntl,subcategory_industrial_enterprises_count_2_hn_0_20_prcntl,subcategory_industrial_enterprises_count_2_hn_20_40_prcntl,subcategory_industrial_enterprises_count_2_hn_40_60_prcntl,subcategory_industrial_enterprises_count_2_hn_60_80_prcntl,subcategory_industrial_enterprises_count_2_hn_80_100_prcntl
0,8911060006bffff,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,89110600077ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,89110600113ffff,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0

In [ ]:
#df_geo_percentile_ARL

**making combined df**

In [200]:
df_yandex_unq_atr = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_yandex_unq_atr.csv')

In [201]:
df_yandex_unq_atr.shape

(3892, 5)

In [202]:
df_yandex_unq_atr.head(3)

,h3_9,total_subcategory_cinemas_neighbour_4_hn_binary,total_subcategory_shopping_centers_neighbour_4_hn_binary,total_subcategory_museums_neighbour_4_hn_binary,total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn_binary
0,8911060006bffff,0.0,1.0,0.0,1.0
1,89110600077ffff,0.0,1.0,0.0,0.0
2,89110600113ffff,0.0,1.0,0.0,0.0


In [ ]:
#df_geo_percentile_ARL
#df_yandex_unq_atr
#df_geo_binary_part

In [203]:
df_geo_ARL_combined = pd.merge(df_geo_percentile_ARL, df_geo_binary_part,
                                how='left', left_on='h3_9', right_on='h3_9')

df_geo_ARL_combined = pd.merge(df_geo_ARL_combined, df_yandex_unq_atr,
                                how='left', left_on='h3_9', right_on='h3_9')

In [206]:
df_geo_percentile_ARL.shape, df_yandex_unq_atr.shape, df_geo_binary_part.shape

((1335, 291), (3892, 5), (1335, 45))

In [204]:
df_geo_ARL_combined.shape

(1335, 339)

In [205]:
df_geo_ARL_combined.head(3)

,h3_9,cian_total_listing_count_including_hex_3hn_0_20_prcntl,cian_total_listing_count_including_hex_3hn_20_40_prcntl,cian_total_listing_count_including_hex_3hn_40_60_prcntl,cian_total_listing_count_including_hex_3hn_60_80_prcntl,cian_total_listing_count_including_hex_3hn_80_100_prcntl,cian_total_area_including_hex_3hn_0_20_prcntl,cian_total_area_including_hex_3hn_20_40_prcntl,cian_total_area_including_hex_3hn_40_60_prcntl,cian_total_area_including_hex_3hn_60_80_prcntl,cian_total_area_including_hex_3hn_80_100_prcntl,cian_median_area_including_hex_3hn_0_20_prcntl,cian_median_area_including_hex_3hn_20_40_prcntl,cian_median_area_including_hex_3hn_40_60_prcntl,cian_median_area_including_hex_3hn_60_80_prcntl,cian_median_area_including_hex_3hn_80_100_prcntl,cian_avg_area_including_hex_3hn_0_20_prcntl,cian_avg_area_including_hex_3hn_20_40_prcntl,cian_avg_area_including_hex_3hn_40_60_prcntl,cian_avg_area_including_hex_3hn_60_80_prcntl,cian_avg_area_including_hex_3hn_80_100_prcntl,cian_median_price_per_m2_including_hex_3hn_0_20_prcntl,cian_median_price_per_m2_including_hex_3hn_20_40_prcntl,cian_median_price_per_m2_including_hex_3hn_40_60_prcntl,cian_median_price_per_m2_including_hex_3hn_60_80_prcntl,cian_median_price_per_m2_including_hex_3hn_80_100_prcntl,cian_avg_price_per_m2_including_hex_3hn_0_20_prcntl,cian_avg_price_per_m2_including_hex_3hn_20_40_prcntl,cian_avg_price_per_m2_including_hex_3hn_40_60_prcntl,cian_avg_price_per_m2_including_hex_3hn_60_80_prcntl,cian_avg_price_per_m2_including_hex_3hn_80_100_prcntl,total_population_neighbour_2_hn_0_20_prcntl,total_population_neighbour_2_hn_20_40_prcntl,total_population_neighbour_2_hn_40_60_prcntl,total_population_neighbour_2_hn_60_80_prcntl,...,category_airports_count_2_hn,category_gymnasiums_count_2_hn,category_colleges_count_2_hn,category_lyceums_count_2_hn,category_planetariums_count_2_hn,category_amusement_parks_count_2_hn,subcategory_universities_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,admiralteysky_district,vasileostrovsky_district,vyborgsky_district,kalininsky_district,kirovsky_district,krasnogvardeysky_district,krasnoselsky_district,moskovsky_district,nevsky_district,petrogradsky_district,primorsky_district,pushkinsky_district,frunzensky_district,tsentralny_district,total_subcategory_cinemas_neighbour_4_hn_binary,total_subcategory_shopping_centers_neighbour_4_hn_binary,total_subcategory_museums_neighbour_4_hn_binary,total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn_binary
0,8911060006bffff,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0
1,89110600077ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0
2,89110600113ffff,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0


# Combine all datasets to one for ARL

**currently withou df geo**

In [207]:
df_ARL_combined = pd.merge(df_hex_total_ARL_combined, df_hex_unique_ARL_combined,
                                how='left', left_on='h3_9', right_on='h3_9')

df_ARL_combined = pd.merge(df_ARL_combined, df_time_ARL_combined,
                                how='left', left_on='h3_9', right_on='h3_9')

df_ARL_combined = pd.merge(df_ARL_combined, df_geo_ARL_combined,
                                how='left', left_on='h3_9', right_on='h3_9')

In [208]:
df_ARL_combined.shape

(1335, 1299)

In [209]:
df_ARL_combined.head(3)

,h3_9,total_visits_0_20_prcntl,total_visits_20_40_prcntl,total_visits_40_60_prcntl,total_visits_60_80_prcntl,total_visits_80_100_prcntl,car_owners_total_0_20_prcntl,car_owners_total_20_40_prcntl,car_owners_total_40_60_prcntl,car_owners_total_60_80_prcntl,car_owners_total_80_100_prcntl,carsharing_user_total_0_20_prcntl,carsharing_user_total_20_40_prcntl,carsharing_user_total_40_60_prcntl,carsharing_user_total_60_80_prcntl,carsharing_user_total_80_100_prcntl,not_citizen_rf_total_0_20_prcntl,not_citizen_rf_total_20_40_prcntl,not_citizen_rf_total_40_60_prcntl,not_citizen_rf_total_60_80_prcntl,not_citizen_rf_total_80_100_prcntl,healthy_food_inter_total_0_20_prcntl,healthy_food_inter_total_20_40_prcntl,healthy_food_inter_total_40_60_prcntl,healthy_food_inter_total_60_80_prcntl,healthy_food_inter_total_80_100_prcntl,public_trnsprt_user_total_0_20_prcntl,public_trnsprt_user_total_20_40_prcntl,public_trnsprt_user_total_40_60_prcntl,public_trnsprt_user_total_60_80_prcntl,public_trnsprt_user_total_80_100_prcntl,pharmacies_inter_total_0_20_prcntl,pharmacies_inter_total_20_40_prcntl,pharmacies_inter_total_40_60_prcntl,pharmacies_inter_total_60_80_prcntl,...,category_airports_count_2_hn,category_gymnasiums_count_2_hn,category_colleges_count_2_hn,category_lyceums_count_2_hn,category_planetariums_count_2_hn,category_amusement_parks_count_2_hn,subcategory_universities_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,admiralteysky_district,vasileostrovsky_district,vyborgsky_district,kalininsky_district,kirovsky_district,krasnogvardeysky_district,krasnoselsky_district,moskovsky_district,nevsky_district,petrogradsky_district,primorsky_district,pushkinsky_district,frunzensky_district,tsentralny_district,total_subcategory_cinemas_neighbour_4_hn_binary,total_subcategory_shopping_centers_neighbour_4_hn_binary,total_subcategory_museums_neighbour_4_hn_binary,total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn_binary
0,8911060006bffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0
1,89110600077ffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0
2,89110600113ffff,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0


In [210]:
path = 'C:/Users/elysi/Tele2_project/data/pre_model_state/df_ARL_combined_share_percentiles_yandex_1299_features_final_binary_df.csv'
df_ARL_combined.to_csv(path, index=False)

# Combine all percentile dataset for better interpretation

In [211]:
df_time_percentile_ranges['base'] = 'df_time'

In [212]:
df_time_percentile_ranges.head(3)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile,base
0,weekday_visits_share,0.329004,0.623243,0.642992,0.654862,0.669048,0.810526,df_time
1,weekend_visits_share,0.189474,0.330952,0.345138,0.357008,0.376757,0.670996,df_time
2,Stay05_1h_share,0.000000,0.036545,0.051308,0.064843,0.082786,0.193548,df_time


In [213]:
df_hex_unique_percentile_ranges['base'] = 'df_hex_unique'

In [214]:
df_hex_unique_percentile_ranges.head(3)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile,base
0,car_owners_unique_share,0.109375,0.418520,0.462931,0.500000,0.541275,0.813953,df_hex_unique
1,carsharing_user_unique_share,0.027778,0.141694,0.172701,0.200000,0.238132,0.413333,df_hex_unique
2,not_citizen_rf_unique_share,0.300000,0.472727,0.500000,0.527974,0.557748,0.707317,df_hex_unique


In [215]:
df_hex_total_percentile_ranges['base'] = 'df_hex_total'

In [216]:
df_hex_total_percentile_ranges.head(3)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile,base
0,car_owners_total_share,0.048485,0.428987,0.493605,0.548413,0.605476,0.891429,df_hex_total
1,carsharing_user_total_share,0.006250,0.135825,0.182488,0.227491,0.274963,0.645833,df_hex_total
2,not_citizen_rf_total_share,0.180055,0.452160,0.508346,0.556226,0.612625,0.937705,df_hex_total


In [217]:
df_geo_percentile_ranges = geo_percentiles.copy()

In [218]:
df_geo_percentile_ranges['base'] = 'df_geo'

In [219]:
df_geo_percentile_ranges.head(3)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile,base
0,cian_total_listing_count_including_hex_3hn,0.0,8.00,16.000,41.00,119.00,470.00,df_geo
1,cian_total_area_including_hex_3hn,0.0,1366.86,4163.880,17833.51,49882.48,288020.97,df_geo
2,cian_median_area_including_hex_3hn,0.0,83.10,115.782,147.00,208.72,2415.00,df_geo


**combine and save**

In [220]:
df_percentiles_combined = pd.concat([df_time_percentile_ranges, df_hex_unique_percentile_ranges], ignore_index=True)

In [221]:
df_percentiles_combined = pd.concat([df_percentiles_combined, df_hex_total_percentile_ranges], ignore_index=True)

In [222]:
df_percentiles_combined = pd.concat([df_percentiles_combined, df_geo_percentile_ranges], ignore_index=True)

In [223]:
df_percentiles_combined.shape

(255, 8)

In [224]:
df_percentiles_combined.head(5)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile,base
0,weekday_visits_share,0.329004,0.623243,0.642992,0.654862,0.669048,0.810526,df_time
1,weekend_visits_share,0.189474,0.330952,0.345138,0.357008,0.376757,0.670996,df_time
2,Stay05_1h_share,0.000000,0.036545,0.051308,0.064843,0.082786,0.193548,df_time
3,Stay1_3h_share,0.274854,0.439047,0.472037,0.513927,0.563195,0.844262,df_time
4,Stay3_6h_share,0.030303,0.146705,0.161178,0.173155,0.185286,0.314356,df_time


In [225]:
path = 'C:/Users/elysi/Tele2_project/data/pre_model_state/df_percentiles_ranges_combined_final_binary_df.csv'
df_percentiles_combined.to_csv(path, index=False)

# Additional Material (Don't use it)

In [394]:
columns_ending_with_count = [col for col in df_geo.columns if col.endswith('_count')]
columns_ending_with_district = [col for col in df_geo.columns if col.endswith('район')]
columns_ending_with_count.extend(columns_ending_with_district)
columns_ending_with_count.append('h3_9')

df_geo_count_dist_osm = df_geo[columns_ending_with_count]


col = df_geo_count_dist_osm.pop('h3_9')
df_geo_count_dist_osm.insert(0, col.name, col)

In [395]:
df_geo_count_dist_osm.head(3)

,h3_9,highway_count,footway_count,apartments_count,detached_count,dormitory_count,hotel_count,house_count,semidetached_house_count,terrace_count,commercial_count,office_count,retail_count,train_station_count,bus_stop_count,crossing_count,cafe_count,fast_food_count,restaurant_count,college_count,language_school_count,school_count,university_count,atm_count,bank_count,clinic_count,hospital_count,pharmacy_count,theatre_count,townhall_count,bench_count,Адмиралтейский район,Василеостровский район,Выборгский район,Калининский район,Кировский район,Красногвардейский район,Красносельский район,Московский район,Невский район,Петроградский район,Приморский район,Пушкинский район,Фрунзенский район,Центральный район
0,8911060006bffff,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,89110600077ffff,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,89110600113ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [396]:
df_geo_count_dist_osm.shape

(1335, 45)

In [397]:
remove_columns = df_geo_count_dist_osm.columns.to_list()

In [398]:
for column in df_geo_count_dist_osm.columns:
    if column not in ['h3_9']:
        df_geo_count_dist_osm.rename(columns={column: column + '_osm'}, inplace=True)

In [399]:
geo_percentiles_short = filter_columns_and_calculate_percentiles(df_geo, remove_columns, [0, 20, 40, 60, 80, 100])
geo_percentiles_short

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,building_counter_osm,0.0,2.0,4.0,6.0,9.0,39.0
1,amenity_counter_osm,0.0,1.0,3.0,8.0,15.0,95.0
2,amount_unqiue_vehicles,0.0,0.0,0.0,1.0,2.0,3.0
3,unique_vehicle_stops,0.0,0.0,0.0,1.0,2.0,8.0
4,unique_stops,0.0,0.0,0.0,1.0,2.0,7.0
5,tram_stop_counter,0.0,0.0,0.0,0.0,0.0,3.0
6,unique_clubs_names,0.0,0.0,0.0,0.0,0.0,6.0
7,unique_clubs_sortes,0.0,0.0,0.0,0.0,0.0,4.0
8,total_fit_clubs,0.0,0.0,0.0,0.0,0.0,6.0


**difficult to do enythink to it, so just lets make it binary, if 0 then 0, if more then 0 then 1**

In [293]:
binary_columns = geo_percentiles_short.column_name.to_list()
binary_columns.append('h3_9')

In [294]:
df_geo_other_columns = df_geo[binary_columns]

col = df_geo_other_columns.pop('h3_9')
df_geo_other_columns.insert(0, col.name, col)
df_geo_other_columns.head(3)

,h3_9,building_counter_osm,amenity_counter_osm,amount_unqiue_vehicles,unique_vehicle_stops,unique_stops,tram_stop_counter,unique_clubs_names,unique_clubs_sortes,total_fit_clubs
0,8911060006bffff,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,89110600077ffff,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,0,3,0.0,0.0,0.0,0.0,4.0,3.0,4.0


In [295]:
binary_columns = geo_percentiles_short.column_name.to_list()
columns_to_replace = binary_columns
df_geo_other_columns[columns_to_replace] = df_geo_other_columns[columns_to_replace].applymap(lambda x: 1 if x > 0 else x)

In [296]:
df_geo_other_columns

,h3_9,building_counter_osm,amenity_counter_osm,amount_unqiue_vehicles,unique_vehicle_stops,unique_stops,tram_stop_counter,unique_clubs_names,unique_clubs_sortes,total_fit_clubs
0,8911060006bffff,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,89110600077ffff,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,0,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0
7,8911060018fffff,1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
9,8911060019bffff,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3878,891106ab6b7ffff,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,891106ab6bbffff,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3882,891106ab6cbffff,1,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3884,891106ab6d7ffff,1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [297]:
for column in df_geo_other_columns.columns:
    if column not in ['h3_9']:
        df_geo_other_columns.rename(columns={column: column + '_binary'}, inplace=True)

# IMPORTANT currently i delete all OSM columns. change when you get normal data

In [298]:
df_geo_other_columns = df_geo_other_columns[[col for col in df_geo_other_columns.columns if 'osm' not in col]]

df_geo_count_dist_osm = df_geo_count_dist_osm[[col for col in df_geo_count_dist_osm.columns if 'район' in col or col == 'h3_9']]

In [299]:
df_geo_other_columns.head(3)

,h3_9,amount_unqiue_vehicles_binary,unique_vehicle_stops_binary,unique_stops_binary,tram_stop_counter_binary,unique_clubs_names_binary,unique_clubs_sortes_binary,total_fit_clubs_binary
0,8911060006bffff,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,89110600077ffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [300]:
df_geo_count_dist_osm.head(3)

,h3_9,Адмиралтейский район_osm,Василеостровский район_osm,Выборгский район_osm,Калининский район_osm,Кировский район_osm,Красногвардейский район_osm,Красносельский район_osm,Московский район_osm,Невский район_osm,Петроградский район_osm,Приморский район_osm,Пушкинский район_osm,Фрунзенский район_osm,Центральный район_osm
0,8911060006bffff,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,89110600077ffff,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,89110600113ffff,0,0,0,0,0,0,0,1,0,0,0,0,0,0
